In [1]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import dateparser
import os

def scrap_novojob():
    # Liste des liens pour chaque catégorie
    categories = [
        "toutes les offres d'emploi",
        
    ]

    base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
    category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    intitules_list = []
    entreprises_list = []
    pays_list = []
    dates_list = []
    niveau_list = []
    experience_list = []

    url_list = []
    all_job_lien = []

    # Parcourir les liens de chaque catégorie
    for category_link in category_links:
        req = requests.get(category_link, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='row-fluid job-details pointer')

        for offre in offres:
            offre_link_tag = offre.find('a')
            if offre_link_tag:
                offre_link = offre_link_tag['href']
                all_job_lien.append(offre_link)

            entreprise_tag = offre.find('h6', class_='ellipsis')
            entreprise = entreprise_tag.get_text().strip() if entreprise_tag else None

            intitule_tag = offre.find('h2', class_='ellipsis row-fluid')
            intitule = intitule_tag.get_text().strip() if intitule_tag else None

            bloc_bottom = offre.find_next('div', class_='bloc-bottom')

            pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
            pays = pays_info.find_parent().text.strip() if pays_info else None

            date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
            date = date_info.find_parent().text.strip() if date_info else None

            niveau_info = bloc_bottom.find('i', class_='fa fa-bookmark icon-left')
            niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

            match = re.match(r'(.+) \((.+)\)', niveau_text)
            niveau_col, experience_col = match.groups() if match else (None, None)

            intitules_list.append(intitule)
            entreprises_list.append(entreprise)
            pays_list.append(pays)
            dates_list.append(date)
            niveau_list.append(niveau_col)
            experience_list.append(experience_col)
            
            url_list.append(category_link)

    # Création du DataFrame
    df_offers = pd.DataFrame({
        'Intitule': intitules_list,
        'Entreprise': entreprises_list,
        'Pays': pays_list,
        'Date': dates_list,
        'Niveau': niveau_list,
        'Experience_lettre': experience_list,
        'URL': url_list,
        'Offre_Link': all_job_lien
    })

    # Importer les détails de chaque offre d'emploi
    job_urls = list(df_offers['Offre_Link'])
    all_job_details = []

    for url in job_urls:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        job_details = {}
        
        # Ajouter le lien
        job_details["Offre_Link"] = url
        
        # Extraction des détails de l'offre d'emploi
        details_section = soup.find('ul', class_='text-small')
        if details_section:
            for li in details_section.find_all('li', class_='row-fluid'):
                key = li.find('span', class_='span4').text.strip()
                value = li.find('span', class_='span8').text.strip()
                job_details[key] = value

        # Extraction du texte fourni
        description_section = soup.find('div', class_='spaced details-description')
        if description_section:
            provided_text = description_section.text.strip()
            job_details['Provided Text'] = provided_text

        all_job_details.append(job_details)

    # Création d'un DataFrame pour les détails des offres d'emploi
    df_Novojob = pd.DataFrame(all_job_details)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Novojob = pd.merge(df_offers,df_Novojob, on='Offre_Link')
    df_Novojob = df_Novojob.drop_duplicates()
    df_Novojob.reset_index(drop=True, inplace=True)

    
    equivalences = {
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Nom de l'entreprise": "RAISON_SOCIALE_DE_L_ENTREPRISE",
    "Secteur d'activité": "BRANCHE_D_ACTIVITE",
    "Lieu de travail": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Date d'expiration": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Nombre de postes": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Niveau de poste": "SPECIALITE",
    "Niveau d'étude (diplome)": "DIPLOME",
    "Type de contrat": "TYPE_DE_CONTRAT_DU_POSTE",
    "Provided Text": "Description",
    "Intitule": "INTITULE_DU_POSTE",
    "Entreprise": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL",
    "Date": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Niveau": "EXPERIENCE_PROFESSIONNELLE",
    "Experience_lettre": None,
    "url_Lien": "Offre_Link1"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    df_Novojob = renommer_colonnes(df_Novojob, equivalences)
    df_Novojob['NOMBRE_DE_POSTES_A_POURVOIR'] = df_Novojob['NOMBRE_DE_POSTES_A_POURVOIR'].str.extract(r"([0-9,]+)")
    df_Novojob[["VILLE_DU_POSTE_DE_TRAVAIL","PAYS_DU_POSTE_DE_TRAVAIL"]] = df_Novojob['LIEU_DU_POSTE_DE_TRAVAIL'].str.split(',', expand=True)
    for i in range(len(df_Novojob["PAYS_DU_POSTE_DE_TRAVAIL"])):
        if df_Novojob["PAYS_DU_POSTE_DE_TRAVAIL"][i] is None :
            df_Novojob.loc[i, "PAYS_DU_POSTE_DE_TRAVAIL"] = df_Novojob["VILLE_DU_POSTE_DE_TRAVAIL"][i]
            df_Novojob.loc[i, "VILLE_DU_POSTE_DE_TRAVAIL"]= " "


    # Réorganiser les colonnes selon vos besoins
    df_Novojob[["EXPERIENCE_PROFESSIONNELLE", 'Unite_EXPERIENCE_PROFESSIONNELLE']] = df_Novojob["Experience_lettre"].str.extract(r"([0-9]+)\s*([a-zA-Z]+)")


    return df_Novojob

df_novojob=scrap_novojob()
df_novojob1=df_novojob
df_novojob

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,SPECIALITE,BRANCHE_D_ACTIVITE,Description,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,DIPLOME,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,Unite_EXPERIENCE_PROFESSIONNELLE
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan,ans
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,Stagiaire / Etudiant,"Informatique, Télécom, Internet",Description de la mission Participation à la r...,01,Stage – Temps partiel,NaN,NaN,,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,01,CDI,"Master 2, Ingéniorat, Bac + 5",NaN,Abidjan,ans
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,02,CDI,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,Débutant / Junior,"Informatique, Télécom, Internet",Dans le cadre de notre participation à un salo...,01,NaN,NaN,NaN,,NaN
5,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,2,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,Débutant / Junior| Confirmé / Expérimenté,Services,OLSY GROUP est une entreprise qui offre des se...,10,CDD,TS Bac +2,NaN,,ans
6,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",04 Juin,Responsable d'équipe| Manager / Responsable dé...,Services,Nous recherchons pour notre client leader dans...,01,CDI| CDD,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans
7,Démonstrateur,Entreprise anonyme,Côte d'ivoire,23 Février,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",23 Mai,Confirmé / Expérimenté,"BTP, Construction, Immobilier",Présentation : Intervenant dans le secteur de ...,01,CDD,Baccalauréat| Formation Professionnelle,Entreprise anonyme,Abidjan,ans
8,Commercial H/F,DOOYA Group,Côte d'ivoire,29 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,27 Juin,Stagiaire / Etudiant,"Distribution, Commerce",DOOYA est une entreprise spécialisée dans le d...,50,Indépendant/Saisonnier,Niveau secondaire| Niveau terminal,NaN,,NaN
9,Stagiaire Technico-Commerciale,HASNET CORPORATION,Côte d'ivoire,15 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Côte d'ivoire| Abidjan, Côte d'ivoire",14 Juillet,Stagiaire / Etudiant,"Distribution, Commerce",Entreprise de sécurité électronique située à Y...,01,NaN,NaN,NaN,Côte d'ivoire| Abidjan,NaN


In [ ]:
Colonnes uniques dans le premier ensemble : {'RAISON_SOCIALE_DE_L_ENTREPRISE', "Niveau d'étude (diplome)", 'Entreprise', 'NOMBRE_DE_POSTES_A_POURVOIR', 'Texte_fourni', 'Niveau', 'Experience_lettre', 'VILLE_DU_POSTE_DE_TRAVAIL', 'Niveau de poste', 'url'}
Colonnes uniques dans le deuxième ensemble : {'Code', 'Unite_EXPERIENCE_PROFESSIONNELLE', 'Date_DEdition', 'EXPERIENCE_PROFESSIONNELLE', 'URL', 'Description', 'SPECIALITE', 'Sous_titre', 'Date_limite', 'DIPLOME'}


In [ ]:
EXPERIENCE_PROFESSIONNELLE

In [107]:
df_final = pd.concat([df_novojob,df_novojob1 ], axis=0, ignore_index=True)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,SPECIALITE,BRANCHE_D_ACTIVITE,Description,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,DIPLOME,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,Unite_EXPERIENCE_PROFESSIONNELLE
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan,ans
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,Stagiaire / Etudiant,"Informatique, Télécom, Internet",Description de la mission Participation à la r...,01,Stage – Temps partiel,NaN,NaN,,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,01,CDI,"Master 2, Ingéniorat, Bac + 5",NaN,Abidjan,ans
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,02,CDI,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,Débutant / Junior,"Informatique, Télécom, Internet",Dans le cadre de notre participation à un salo...,01,NaN,NaN,NaN,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PMO en Gestion de Projets Régionaux et Gestion...,Société Générale Afrique de l'ouest ( Centre ...,Côte d'ivoire,22 Février,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Mai,Débutant / Junior| Stagiaire / Etudiant,"Banque, Assurance, Finance",Société Générale Afrique de l'Ouest recherche ...,01,Stage – Temps partiel,"Master 1, Licence Bac + 4| Master 2, Ingénior...",NaN,Abidjan,NaN
96,Chef de Projet pour l'Intégration des solution...,Ascens,Côte d'ivoire,15 Avril,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",14 Juillet,Confirmé / Expérimenté,"Informatique, Télécom, Internet",Présentation de l'entreprise : ASCENS Service...,01,CDI,"Master 1, Licence Bac + 4| Master 2, Ingénior...",NaN,Abidjan,ans
97,"Conseiller.ère Crise, Conflits et Catastrophes...",Giz Côte d'Ivoire,Côte d'ivoire,18 Avril,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",09 Mai,Confirmé / Expérimenté,Services,"Poste basé à KorhogoContexte :La GIZ, Deutsche...",01,CDD,"Master 2, Ingéniorat, Bac + 5",NaN,Abidjan,ans
98,Comptable Confirmé,Prosuma,Côte d'ivoire,12 Avril,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",11 Juillet,Confirmé / Expérimenté,"Distribution, Commerce",Pour accompagner la croissance de notre activi...,02,CDI| CDD,"Licence (LMD), Bac + 3",NaN,Abidjan,ans


In [3]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    # Convertir le texte en minuscules
    text = text.lower()
    # Enlever la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Prétraitement des données
df_final['Texte_clean'] = df_final['Description'].apply(preprocess_text)

# Vectorisation des données
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Texte_clean'])

# Calcul de similarité
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Seuil de similarité
threshold = 0.9

# Identification des doublons
duplicates = set()
for i in range(len(similarities)):
    for j in range(i+1, len(similarities)):
        if similarities[i,j] > threshold:
            duplicates.add(j)

# Suppression des doublons
df_final = df_final.drop(index=duplicates).reset_index(drop=True)

# Afficher ou retourner le DataFrame final
df_final


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,SPECIALITE,BRANCHE_D_ACTIVITE,Description,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,DIPLOME,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,Unite_EXPERIENCE_PROFESSIONNELLE,Texte_clean
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan,ans,entreprise spécialisée dans l’ingénierie mécan...
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,Stagiaire / Etudiant,"Informatique, Télécom, Internet",Description de la mission Participation à la r...,01,Stage – Temps partiel,NaN,NaN,,NaN,description de la mission participation à la r...
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,01,CDI,"Master 2, Ingéniorat, Bac + 5",NaN,Abidjan,ans,sous la responsabilité du responsable techniqu...
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,Responsable d'équipe,Services,Sous la responsabilité du Responsable Techniqu...,02,CDI,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans,sous la responsabilité du responsable techniqu...
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,Débutant / Junior,"Informatique, Télécom, Internet",Dans le cadre de notre participation à un salo...,01,NaN,NaN,NaN,,NaN,dans le cadre de notre participation à un salo...
5,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,2,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,Débutant / Junior| Confirmé / Expérimenté,Services,OLSY GROUP est une entreprise qui offre des se...,10,CDD,TS Bac +2,NaN,,ans,olsy group est une entreprise qui offre des se...
6,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",04 Juin,Responsable d'équipe| Manager / Responsable dé...,Services,Nous recherchons pour notre client leader dans...,01,CDI| CDD,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans,nous recherchons pour notre client leader dans...
7,Démonstrateur,Entreprise anonyme,Côte d'ivoire,23 Février,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",23 Mai,Confirmé / Expérimenté,"BTP, Construction, Immobilier",Présentation : Intervenant dans le secteur de ...,01,CDD,Baccalauréat| Formation Professionnelle,Entreprise anonyme,Abidjan,ans,présentation intervenant dans le secteur de l...
8,Commercial H/F,DOOYA Group,Côte d'ivoire,29 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,27 Juin,Stagiaire / Etudiant,"Distribution, Commerce",DOOYA est une entreprise spécialisée dans le d...,50,Indépendant/Saisonnier,Niveau secondaire| Niveau term

In [72]:
print(tfidf_matrix)

  (0, 20)	0.05066154501573576
  (0, 373)	0.034213683600243995
  (0, 1529)	0.034213683600243995
  (0, 764)	0.03608752454166718
  (0, 2292)	0.05066154501573576
  (0, 1033)	0.02781389806449507
  (0, 1712)	0.044931628446076434
  (0, 1516)	0.03833844205535161
  (0, 374)	0.05066154501573576
  (0, 1691)	0.05066154501573576
  (0, 248)	0.044931628446076434
  (0, 2726)	0.02601533909496746
  (0, 1660)	0.04115742888726687
  (0, 2403)	0.044931628446076434
  (0, 2871)	0.05066154501573576
  (0, 2842)	0.0299569592499855
  (0, 382)	0.019330209798413815
  (0, 1739)	0.05066154501573576
  (0, 599)	0.04115742888726687
  (0, 2136)	0.024465710656735654
  (0, 2657)	0.021329615167153534
  (0, 2280)	0.05066154501573576
  (0, 925)	0.044931628446076434
  (0, 906)	0.031204574052254914
  (0, 556)	0.044931628446076434
  :	:
  (99, 1362)	0.0987670631533543
  (99, 1186)	0.027253239808205477
  (99, 870)	0.0375501607685741
  (99, 2618)	0.10083977609938204
  (99, 2717)	0.034301219280358015
  (99, 2257)	0.0450402267715862

In [75]:
similarities.shape

(100, 100)

In [54]:
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,INTITULE_DU_POSTE_clean,Texte_clean,Entreprise_clean,Niveau_etude_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,Abidjan,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...
1,Commercial Terrain,KOMIAN AI,Côte d'ivoire,02 Février,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,Aujourd'hui,...,25,Consultant,NaN,,commercial terrain,komian ia est une entreprise de technologie d’...,komian ai,ts bac 2 licence lmd bac 3,consultant,commercial terrain komian ia est une entrepris...
2,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,Stagiaire / Etudiant,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,01,Stage – Temps partiel,NaN,,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...
3,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,01,CDI,NaN,Abidjan,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...
4,Chargé Administratif RH et Commercial Adjoint,Exceliam,Côte d'ivoire,07 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",05 Mai,...,01,CDI| CDD,NaN,Abidjan,chargé administratif rh et commercial adjoint,sous le contrôle direct du responsable de l’ad...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi cdd,chargé administratif rh et commercial adjoint ...
5,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,Responsable d'équipe,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,02,CDI,NaN,Abidjan,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...
6,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,Débutant / Junior,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,01,NaN,NaN,,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...
7,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,Confirmé / Expérimenté,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,...,10,CDD,NaN,,télévendeur hf,olsy group est une entreprise qui offre des se...,olsy group,ts bac 2,cdd,télévendeur hf olsy group est une entreprise q...
8,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte 

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Prétraitement des données
df_final['Description_clean'] = df_final['Description'].apply(preprocess_text)
df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].apply(preprocess_text)

# Concaténer toutes les variables pour créer un texte combiné
df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean']+ ' ' + df_final['Description_clean'] 

# Vectorisation des données
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

# Calcul de similarité
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Seuil de similarité
threshold = 0.9

# Identification des doublons
duplicates = set()
for i in range(len(similarities)):
    for j in range(i+1, len(similarities)):
        if similarities[i,j] > threshold:
            duplicates.add(j)

# Suppression des doublons
df_final = df_final.drop(index=duplicates).reset_index(drop=True)

# Afficher ou retourner le DataFrame final
df_final


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,DIPLOME,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,Unite_EXPERIENCE_PROFESSIONNELLE,Texte_clean,Description_clean,INTITULE_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,Abidjan,ans,entreprise spécialisée dans l’ingénierie mécan...,entreprise spécialisée dans l’ingénierie mécan...,gestionnaire des ressources humaines,gestionnaire des ressources humaines entrepris...
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,01,Stage – Temps partiel,NaN,NaN,,NaN,description de la mission participation à la r...,description de la mission participation à la r...,stagiaire réseau fibre optique et informatique,stagiaire réseau fibre optique et informatique...
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,01,CDI,"Master 2, Ingéniorat, Bac + 5",NaN,Abidjan,ans,sous la responsabilité du responsable techniqu...,sous la responsabilité du responsable techniqu...,chiffreur btp,chiffreur btp sous la responsabilité du respon...
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,02,CDI,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans,sous la responsabilité du responsable techniqu...,sous la responsabilité du responsable techniqu...,chargé daffaires senior,chargé daffaires senior sous la responsabilité...
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,01,NaN,NaN,NaN,,NaN,dans le cadre de notre participation à un salo...,dans le cadre de notre participation à un salo...,assistante anglais et français pour un salon d...,assistante anglais et français pour un salon d...
5,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,2,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,...,10,CDD,TS Bac +2,NaN,,ans,olsy group est une entreprise qui offre des se...,olsy group est une entreprise qui offre des se...,télévendeur hf,télévendeur hf olsy group est une entreprise q...
6,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",04 Juin,...,01,CDI| CDD,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans,nous recherchons pour notre client leader dans...,nous recherchons pour notre client leader dans...,responsable commercial,responsable commercial nous recherchons pour n...
7,Démonstrateur,Entreprise anonyme,Côte d'ivoire,23 Février,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",23 Mai,...,01,CDD,Baccalauréat| Formation Professionnelle,Entreprise anonyme,Abidjan,ans,présentation intervenant dans le secteur de l...,présentation intervenant dans le secteur de l...,démonstrateur,démonstrateur présentation intervenant dans l...
8,Commercial H/F,DOOYA Group,Côte d'ivoire,29 

In [108]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,DIPLOME,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL,Unite_EXPERIENCE_PROFESSIONNELLE,INTITULE_DU_POSTE_clean,Description_clean,Entreprise_clean,DIPLOME_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,Abidjan,ans,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,,NaN,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,"Master 2, Ingéniorat, Bac + 5",NaN,Abidjan,ans,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,,NaN,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...
5,Télévendeur H/F,OLSY GROUP,Côte d'ivoire,03 Avril,2,1 à 2 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juillet,...,TS Bac +2,NaN,,ans,télévendeur hf,olsy group est une entreprise qui offre des se...,olsy group,ts bac 2,cdd,télévendeur hf olsy group est une entreprise q...
6,Responsable Commercial,Exceliam,Côte d'ivoire,06 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",04 Juin,...,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",NaN,Abidjan,ans,responsable commercial,nous recherchons pour notre client leader dans...,exceliam,licence lmd bac 3 master 1 licence bac 4 ma...,cdi cdd,responsable commercial nous recherchons pour n...
7,Démonstrateur,Entreprise anonyme,Côte d'ivoire,23 Février,5,3 à 5 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",23 Mai,...,Baccalauréat| Formation Professionnelle,Entreprise anonyme,Abidjan,ans,démonstrateur,présentation intervenant dans le secteur de l...,entreprise anonyme,baccalauréat formation professionnelle,cdd,démonstrateur présentation intervenant dans l...
8,Commercial H/F,DOOYA Group,Côte d'ivoire,29 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,27 Juin,...,Niveau secondaire| Niveau termi

In [ ]:
tfidf_matrix

In [52]:
help(tfidf_matrix)

Help on csr_matrix in module scipy.sparse._csr object:

class csr_matrix(scipy.sparse._matrix.spmatrix, _csr_base)
 |  csr_matrix(arg1, shape=None, dtype=None, copy=False)
 |  
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_array(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_array(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_array((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_array((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_array((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their


In [11]:
similarities

array([[1.        , 0.34764986, 0.26202349, ..., 0.21265302, 0.43465652,
        0.32698244],
       [0.34764986, 1.        , 0.21974334, ..., 0.25099532, 0.34713357,
        0.31949761],
       [0.26202349, 0.21974334, 1.        , ..., 0.15843724, 0.24178448,
        0.19605545],
       ...,
       [0.21265302, 0.25099532, 0.15843724, ..., 1.        , 0.23132896,
        0.27717748],
       [0.43465652, 0.34713357, 0.24178448, ..., 0.23132896, 1.        ,
        0.32074343],
       [0.32698244, 0.31949761, 0.19605545, ..., 0.27717748, 0.32074343,
        1.        ]])

In [50]:
v.shape

(9220,)

In [41]:
similarities

array([[1.        , 0.35519365, 0.25831521, ..., 0.2113441 , 0.44008386,
        0.33125969],
       [0.35519365, 1.        , 0.21828934, ..., 0.24996177, 0.35457366,
        0.3274887 ],
       [0.25831521, 0.21828934, 1.        , ..., 0.16536667, 0.23795617,
        0.1950946 ],
       ...,
       [0.2113441 , 0.24996177, 0.16536667, ..., 1.        , 0.22464974,
        0.27218893],
       [0.44008386, 0.35457366, 0.23795617, ..., 0.22464974, 1.        ,
        0.32825131],
       [0.33125969, 0.3274887 , 0.1950946 , ..., 0.27218893, 0.32825131,
        1.        ]])

In [6]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import urllib3
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import requests

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By


import dateparser


def emploi_educarriere():
    # Ignorer les avertissements SSL
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    # Fonction pour extraire le texte d'un élément HTML
    def extract_text(element, class_name=None, style=None, text_contains=None):
        if element:
            tag = element.find(class_=class_name, style=style, text=text_contains)
            return tag.text.strip() if tag else ""
        else:
            return ""

    # Fonction pour nettoyer le texte
    def clean_text(text):
        if text is not None:
            cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
            return cleaned_text.strip() if cleaned_text else None
        else:
            return None

    # Fonction pour extraire la date à partir d'un élément HTML
    def extract_date(element, text_contains):
        date_elements = element.find_all('a', class_='text')
        date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
        return date

    # Fonction pour extraire les détails des offres d'emploi
    def scrap_job_details(urls):
        # En-tête pour éviter d'être bloqué
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        options = webdriver.ChromeOptions()
        #options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
        #options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
        chrome_driver_path = 'C:\\Program Files (x86)\\chromedriver.exe'
        options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
        options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
        driver = webdriver.Chrome(options=options)

        # Liste pour stocker les détails de chaque emploi
        all_job_details = []

        # Parcourir les liens
        for url in urls:
            req = requests.get(url, headers=headers)
            soup = BeautifulSoup(req.text, 'html.parser')
            time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

            offres = soup.find_all('div', class_='box row')

            # Parcourir les offres d'emploi sur la page principale
            for offre in offres:
                # Trouver la balise <h4> dans la structure HTML pour extraire le lien
                offre_link_tag = offre.find('h4')

                # Vérifier si la balise <h4> a été trouvée
                if offre_link_tag:
                    # Extraire le lien de l'attribut 'href'
                    offre_link = offre_link_tag.find('a')['href']
                    all_job_details.append({'Offre_Link': offre_link})

        # Fermer le pilote Selenium à la fin
        driver.quit()

        # Concaténer tous les détails des emplois en un seul DataFrame
        if all_job_details:
            all_job_details_df = pd.DataFrame(all_job_details)
            return all_job_details_df
        else:
            print("Aucun détail d'offre d'emploi trouvé.")
            return None

    # Fonction pour récupérer les données des offres d'emploi
    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=200)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='box row')

        data_list = []

        for wrapper in job_description_wrappers:
            h4_tag = wrapper.find('h4')
            poste = extract_text(h4_tag)

            entry_title_tag = wrapper.find('p', class_='entry-title')
            sous_titre = extract_text(entry_title_tag)

            a_text_tag = wrapper.find('a', class_='text')
            code = extract_text(a_text_tag, style='color:#FF0000;')

            date_edition = extract_date(wrapper, "Date d'édition:")
            date_limite = extract_date(wrapper, "Date limite:")

            pays_tag = wrapper.find('a', class_='text')
            pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

            sous_titre = clean_text(sous_titre)

            data_list.append({
                'Poste': clean_text(poste),
                'Sous_titre': sous_titre,
                'Code': clean_text(code),
                'Date_DEdition': date_edition,
                'Date_limite': date_limite,
                'Pays': clean_text(pays),
                'URL': url  # Ajout de la colonne 'URL'
            })

        df = pd.DataFrame(data_list)
        return df

    # Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
    def add_offre_link_column(result_df):
        job_details_df = scrap_job_details(list(result_df["URL"]))
        if job_details_df is not None:
            # Join des DataFrames
            result_df = pd.concat([result_df, job_details_df], axis=1)
            return result_df
        else:
            print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
            return None

    # Fonction pour extraire les informations de l'offre d'emploi
    def extract_job_information(soup, url):
        try:
            # Extraction des informations de l'offre d'emploi
            poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
            type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
            metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
            niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
            experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
            lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
            
            # Extraction des dates de publication et de limite
            date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
            date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
            
            # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
            description = soup.select_one('div.entry-content').text.strip()

            return {
                "Poste": [poste],
                "Type d'offre": [type_offre],
                "Métier(s)": [metiers],
                "Niveau(x)": [niveaux],
                "Expérience": [experience],
                "Lieu": [lieu],
                "Offre_Link": [url],
                "Date de publication": [date_publication],
                "Date limite": [date_limite],
                "Description": [description]
            }
        except Exception as e:
            print(f"An error occurred while extracting job information for URL {url}: {e}")
            # Retourner un dictionnaire avec l'URL en cas d'erreur
            return {"Offre_Link": [url]}

    # Liste des liens
    urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(1)]

    # Créer un DataFrame à partir des liens
    result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

    # Ajouter la colonne 'Offre_Link'
    result_df = add_offre_link_column(result_df)
    # Supprimer les lignes avec plus de 80% de valeurs NaN
    threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
    result_df = result_df.dropna(thresh=threshold)
    result_df = result_df.reset_index(drop=True)
    
    # Liste des URLs à scraper
    urls = list(result_df['Offre_Link'])

    # Liste pour stocker les DataFrames
    dfs = []

    # Boucle sur chaque URL
    for url in urls:
        try:
            # Envoyer une requête GET au site avec un délai de 120 secondes
            response = requests.get(url, headers=headers, verify=True, timeout=120)

            # Vérifier si la requête a réussi (statut 200)
            if response.status_code == 200:
                # Analyser le contenu de la page avec BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup, url)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            else:
                print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
                # Ajouter une ligne avec l'URL en cas d'erreur
                dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        except requests.exceptions.Timeout:
            print(f"Timeout lors de la requête pour l'URL {url}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))
        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
        time.sleep(5)

    # Concaténer tous les DataFrames en un seul DataFrame
    df_Educarriere = pd.concat(dfs, ignore_index=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Educarriere = pd.merge(df_Educarriere, result_df, on='Offre_Link')
    df_Educarriere = df_Educarriere.drop_duplicates()
    df_Educarriere.reset_index(drop=True, inplace=True)
    
    equivalences = {
    "Poste_y": "INTITULE_DU_POSTE",
    "Type d'offre": "TYPE_DE_CONTRAT_DU_POSTE",
    "Métier(s)": "SPECIALITE",
    "Niveau(x)": "DIPLOME",
    "Expérience": "EXPERIENCE_PROFESSIONNELLE",
    "Lieu": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Date de publication": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Date limite": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Poste_x": "BRANCHE_D_ACTIVITE",
    "Description": None,
    "Sous_titre": None,
    "Code": None,
    "Date_DEdition": None,
    "Date_limite": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    
    # Renommer les colonnes du DataFrame
    df_Educarriere = renommer_colonnes(df_Educarriere, equivalences)
    df_Educarriere[["EXPERIENCE_PROFESSIONNELLE", 'Unite_EXPERIENCE_PROFESSIONNELLE']] = df_Educarriere["EXPERIENCE_PROFESSIONNELLE"].str.extract(r"([0-9]+)\s*([a-zA-Z]+)")


    # Réorganiser les colonnes selon vos besoins
    # Vous pouvez réorganiser les colonnes ici

    # Afficher ou retourner le DataFrame selon votre besoin
    return df_Educarriere

df_educarriere = emploi_educarriere()
df_educarriere


,BRANCHE_D_ACTIVITE,TYPE_DE_CONTRAT_DU_POSTE,SPECIALITE,DIPLOME,EXPERIENCE_PROFESSIONNELLE,LIEU_DU_POSTE_DE_TRAVAIL,SITE_WEB_DE_L_ENTREPRISE,DATE_DE_DEBUT_DE_L_OFFRE,DATE_D_EXPIRATION_DE_L_OFFRE,Description,INTITULE_DU_POSTE,Sous_titre,Code,Date_DEdition,Date_limite,PAYS_DU_POSTE_DE_TRAVAIL,URL,Unite_EXPERIENCE_PROFESSIONNELLE
0,RESPONSABLE DE SITE,Emploi,"Hôtellerie/Restauration, Tourisme/Loisirs","BAC+2, BAC+3",3,Cocody,https://emploi.educarriere.ci/offre-117182-res...,06/05/2024,10/05/2024,C360°recruteRESPONSABLE DE SITE\n \n \nDescrip...,COMPTABLE,Type de contrat : CDI si essai concluant Poste...,113006,06/05/2024,18/05/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,ans
1,INFORMATICIEN,Stage,Informatique,BAC+2,1,RIVIERA ATTOABAN,https://emploi.educarriere.ci/offre-117181-inf...,06/05/2024,20/05/2024,EVEN MEDIA INTERACTIVErecruteINFORMATICIEN\n \...,RESPONSABLE DE SITE,C360°recruteRESPONSABLE DE SITEDescription du ...,113005,06/05/2024,10/05/2024,Cocody,https://emploi.educarriere.ci/nos-offres?page1...,ans
2,DEVELOPPEUR,Emploi,Informatique,"BAC+4, BAC+5",5,ABIDJAN,https://emploi.educarriere.ci/offre-117180-dev...,06/05/2024,20/05/2024,ENTREPRISErecruteDEVELOPPEUR\n \n \nDescriptio...,INFORMATICIEN,EVEN MEDIA INTERACTIVErecruteINFORMATICIENDesc...,113004,06/05/2024,20/05/2024,A...,https://emploi.educarriere.ci/nos-offres?page1...,ans
3,UN(1) TEAM LEAD  BUSINESS DEVELOPPER (H/F),Emploi,"Informatique, Télécommunications","BAC+3, BAC+4",3,Côte d'Ivoire,https://emploi.educarriere.ci/offre-117178-un-...,06/05/2024,15/05/2024,ROSAPARKSrecruteUN(1) TEAM LEAD – BUSINESS DEV...,DEVELOPPEUR,ENTREPRISErecruteDEVELOPPEURDescription du pos...,113003,06/05/2024,20/05/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,ans
4,COMMERCIAL BIOMEDICAL (TECHNICO-COMMERCIAL MAT...,Emploi,"Commerce/Ventes, Marketing","BAC+3, BAC+4",3,Abidjan,https://emploi.educarriere.ci/offre-117176-com...,06/05/2024,20/05/2024,CAGRHrecruteCOMMERCIAL BIOMEDICAL (TECHNICO-CO...,UN(1) TEAM LEAD  BUSINESS DEVELOPPER (H/F),ROSAPARKSrecruteUN(1) TEAM LEAD – BUSINESS DEV...,113001,06/05/2024,15/05/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,ans
5,AGENT SUIVI EMPOTAGE (H/F),Emploi,"Logistique/Transport , Transit/Transport","BAC+2, BAC+3",3,Abidjan,https://emploi.educarriere.ci/offre-117175-age...,06/05/2024,20/05/2024,INDUSTRIE SPÉCIALISÉE DANS L'EXPORTATION DU CA...,COMMERCIAL BIOMEDICAL (TECHNICO-COMMERCIAL MAT...,CAGRHrecruteCOMMERCIAL BIOMEDICAL (TECHNICO-CO...,112999,06/05/2024,20/05/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
6,COMMERCIAL NUMERIQUE,Emploi,"Commerce/Ventes, Marketing","BAC+3, BAC+4",3,Abidjan,https://emploi.educarriere.ci/offre-117174-com...,06/05/2024,20/05/2024,CAGRHrecruteCOMMERCIAL NUMERIQUE\n \n \nDescri...,AGENT SUIVI EMPOTAGE (H/F),INDUSTRIE SPÉCIALISÉE DANS L'EXPORTATION DU CA...,112998,06/05/2024,20/05/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
7,AGENT DE CENTRE D'APPELS,Emploi,"Commerce/Ventes, Communication, Télécommunicat...",BAC+2,1,ABIDJAN,https://emploi.educarriere.ci/offre-117173-age...,06/05/2024,30/08/2024,HSD HUMAN RESOURCE SOLUTIONSrecruteAGENT DE CE...,COMMERCIAL NUMERIQUE,CAGRHrecruteCOMMERCIAL NUMERIQUEDescription du...,112997,06/05/2024,20/05/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
8,APPORTEUR D'AFFAIRES,Consultance,Commerce/Ventes,BAC+2,NaN,Côte d'Ivoire,https://emploi.educarriere.ci/offre-117172-app...,06/05/2024,30/05/2024,C360°recruteAPPORTEUR D'AFFAIRES\n \n \nDescri...,AGENT DE CENTRE D'APPELS,HSD HUMAN RESOURCE SOLUTIONSrecruteAGENT DE CE...,112996,06/05/2024,30/08/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,NaN
9,ASSISTANT(E) SOCIAL (E) PRESTATAIRE,Emploi,"Sciences sociales, Sociologie",BAC+2,2,Un centre de santé du district sanitaire de Bo...,https://emploi.educarriere.ci/offre-117171-ass...,06/05/2024,13/05/2024,Health Alliance Côte d'Ivoire (HA-CI)recruteAS...,APPORTEUR D'AFFAI

In [109]:
df_educarriere

,BRANCHE_D_ACTIVITE,TYPE_DE_CONTRAT_DU_POSTE,SPECIALITE,DIPLOME,EXPERIENCE_PROFESSIONNELLE,LIEU_DU_POSTE_DE_TRAVAIL,SITE_WEB_DE_L_ENTREPRISE,DATE_DE_DEBUT_DE_L_OFFRE,DATE_D_EXPIRATION_DE_L_OFFRE,Description,INTITULE_DU_POSTE,Sous_titre,Code,Date_DEdition,Date_limite,PAYS_DU_POSTE_DE_TRAVAIL,URL,Unite_EXPERIENCE_PROFESSIONNELLE
0,RESPONSABLE DE SITE,Emploi,"Hôtellerie/Restauration, Tourisme/Loisirs","BAC+2, BAC+3",3,Cocody,https://emploi.educarriere.ci/offre-117182-res...,06/05/2024,10/05/2024,C360°recruteRESPONSABLE DE SITE\n \n \nDescrip...,COMPTABLE,Type de contrat : CDI si essai concluant Poste...,113006,06/05/2024,18/05/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,ans
1,INFORMATICIEN,Stage,Informatique,BAC+2,1,RIVIERA ATTOABAN,https://emploi.educarriere.ci/offre-117181-inf...,06/05/2024,20/05/2024,EVEN MEDIA INTERACTIVErecruteINFORMATICIEN\n \...,RESPONSABLE DE SITE,C360°recruteRESPONSABLE DE SITEDescription du ...,113005,06/05/2024,10/05/2024,Cocody,https://emploi.educarriere.ci/nos-offres?page1...,ans
2,DEVELOPPEUR,Emploi,Informatique,"BAC+4, BAC+5",5,ABIDJAN,https://emploi.educarriere.ci/offre-117180-dev...,06/05/2024,20/05/2024,ENTREPRISErecruteDEVELOPPEUR\n \n \nDescriptio...,INFORMATICIEN,EVEN MEDIA INTERACTIVErecruteINFORMATICIENDesc...,113004,06/05/2024,20/05/2024,A...,https://emploi.educarriere.ci/nos-offres?page1...,ans
3,UN(1) TEAM LEAD  BUSINESS DEVELOPPER (H/F),Emploi,"Informatique, Télécommunications","BAC+3, BAC+4",3,Côte d'Ivoire,https://emploi.educarriere.ci/offre-117178-un-...,06/05/2024,15/05/2024,ROSAPARKSrecruteUN(1) TEAM LEAD – BUSINESS DEV...,DEVELOPPEUR,ENTREPRISErecruteDEVELOPPEURDescription du pos...,113003,06/05/2024,20/05/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,ans
4,COMMERCIAL BIOMEDICAL (TECHNICO-COMMERCIAL MAT...,Emploi,"Commerce/Ventes, Marketing","BAC+3, BAC+4",3,Abidjan,https://emploi.educarriere.ci/offre-117176-com...,06/05/2024,20/05/2024,CAGRHrecruteCOMMERCIAL BIOMEDICAL (TECHNICO-CO...,UN(1) TEAM LEAD  BUSINESS DEVELOPPER (H/F),ROSAPARKSrecruteUN(1) TEAM LEAD – BUSINESS DEV...,113001,06/05/2024,15/05/2024,d'I...,https://emploi.educarriere.ci/nos-offres?page1...,ans
5,AGENT SUIVI EMPOTAGE (H/F),Emploi,"Logistique/Transport , Transit/Transport","BAC+2, BAC+3",3,Abidjan,https://emploi.educarriere.ci/offre-117175-age...,06/05/2024,20/05/2024,INDUSTRIE SPÉCIALISÉE DANS L'EXPORTATION DU CA...,COMMERCIAL BIOMEDICAL (TECHNICO-COMMERCIAL MAT...,CAGRHrecruteCOMMERCIAL BIOMEDICAL (TECHNICO-CO...,112999,06/05/2024,20/05/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
6,COMMERCIAL NUMERIQUE,Emploi,"Commerce/Ventes, Marketing","BAC+3, BAC+4",3,Abidjan,https://emploi.educarriere.ci/offre-117174-com...,06/05/2024,20/05/2024,CAGRHrecruteCOMMERCIAL NUMERIQUE\n \n \nDescri...,AGENT SUIVI EMPOTAGE (H/F),INDUSTRIE SPÉCIALISÉE DANS L'EXPORTATION DU CA...,112998,06/05/2024,20/05/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
7,AGENT DE CENTRE D'APPELS,Emploi,"Commerce/Ventes, Communication, Télécommunicat...",BAC+2,1,ABIDJAN,https://emploi.educarriere.ci/offre-117173-age...,06/05/2024,30/08/2024,HSD HUMAN RESOURCE SOLUTIONSrecruteAGENT DE CE...,COMMERCIAL NUMERIQUE,CAGRHrecruteCOMMERCIAL NUMERIQUEDescription du...,112997,06/05/2024,20/05/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
8,APPORTEUR D'AFFAIRES,Consultance,Commerce/Ventes,BAC+2,NaN,Côte d'Ivoire,https://emploi.educarriere.ci/offre-117172-app...,06/05/2024,30/05/2024,C360°recruteAPPORTEUR D'AFFAIRES\n \n \nDescri...,AGENT DE CENTRE D'APPELS,HSD HUMAN RESOURCE SOLUTIONSrecruteAGENT DE CE...,112996,06/05/2024,30/08/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,NaN
9,ASSISTANT(E) SOCIAL (E) PRESTATAIRE,Emploi,"Sciences sociales, Sociologie",BAC+2,2,Un centre de santé du district sanitaire de Bo...,https://emploi.educarriere.ci/offre-117171-ass...,06/05/2024,13/05/2024,Health Alliance Côte d'Ivoire (HA-CI)recruteAS...,APPORTEUR D'AFFAI

In [111]:
df_final = pd.concat([df_final,df_educarriere ], axis=0, ignore_index=True)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,INTITULE_DU_POSTE_clean,Description_clean,Entreprise_clean,DIPLOME_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text,Sous_titre,Code,Date_DEdition,Date_limite
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,ASSISTANT(E) SOCIAL (E) PRESTATAIRE,NaN,centre...,06/05/2024,4,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117170-cou...,Abidjan,30/08/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,Health Alliance Côte d'Ivoire (HA-CI)recruteAS...,112994,06/05/2024,13/05/2024
75,COURTIER EN ASSURANCE,NaN,Abidjan,06/05/2024,2,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117169-tel...,Abidjan,11/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,HSD HUMAN RESOURCE SOLUTIONSrecruteCOURTIER EN...,112993,06/05/2024,30/08/2024
76,TELE VENDEUR,NaN,Abidjan,06/05/2024,3,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117168-com...,"Abidjan (Yopougon), Côte dIvoire",15/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,POK ServicesrecruteTELE VENDEURDescription du ...,112992,06/05/2024,11/05/2024
77,COMPTABLE,NaN,(...,06/05/2024,1,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117167-bar...,Abidjan,20/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,DOMAFRIKrecruteCOMPTABLEDescription du posteDO...,112991,06/05/2024,15/05/2024


In [112]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,INTITULE_DU_POSTE_clean,Description_clean,Entreprise_clean,DIPLOME_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Combined_text,Sous_titre,Code,Date_DEdition,Date_limite
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,gestionnaire des ressources humaines,entreprise spécialisée dans l’ingénierie mécan...,,,,gestionnaire des ressources humaines entrepris...,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,stagiaire réseau fibre optique et informatique,description de la mission participation à la r...,wandoo technologies,,stage – temps partiel,stagiaire réseau fibre optique et informatique...,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,chiffreur btp,sous la responsabilité du responsable techniqu...,exceliam,master 2 ingéniorat bac 5,cdi,chiffreur btp sous la responsabilité du respon...,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,chargé daffaires senior,sous la responsabilité du responsable techniqu...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,chargé daffaires senior sous la responsabilité...,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,assistante anglais et français pour un salon d...,dans le cadre de notre participation à un salo...,futuretech,,,assistante anglais et français pour un salon d...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,ASSISTANT(E) SOCIAL (E) PRESTATAIRE,NaN,centre...,06/05/2024,4,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117170-cou...,Abidjan,30/08/2024,...,assistante social e prestataire,hsd human resource solutionsrecrutecourtier en...,,bac3,emploi,assistante social e prestataire hsd human reso...,Health Alliance Côte d'Ivoire (HA-CI)recruteAS...,112994,06/05/2024,13/05/2024
59,COURTIER EN ASSURANCE,NaN,Abidjan,06/05/2024,2,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117169-tel...,Abidjan,11/05/2024,...,courtier en assurance,pok servicesrecrutetele vendeur\n \n \ndescrip...,,bac,stage,courtier en assurance pok servicesrecrutetele ...,HSD HUMAN RESOURCE SOLUTIONSrecruteCOURTIER EN...,112993,06/05/2024,30/08/2024
60,TELE VENDEUR,NaN,Abidjan,06/05/2024,3,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117168-com...,"Abidjan (Yopougon), Côte dIvoire",15/05/2024,...,tele vendeur,domafrikrecrutecomptable\n \n \ndescription du...,,bac3 bac4,emploi,tele vendeur domafrikrecrutecomptable\n \n \nd...,POK ServicesrecruteTELE VENDEURDescription du ...,112992,06/05/2024,11/05/2024
61,COMPTABLE,NaN,(...,06/05/2024,1,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117167-bar...,Abidjan,20/05/2024,...,comptable,potentiel innov rhrecrutebarman hf\n \n \ndesc...,,bac,emploi,comptable potentiel innov rhrecrutebarman hf\n...,DOMAFRIKrecruteCOMPTABLEDescription du poste

In [92]:
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Description_clean,INTITULE_DU_POSTE_clean,Combined_text,Entreprise_clean,DIPLOME_clean,TYPE_DE_CONTRAT_DU_POSTE_clean,Sous_titre,Code,Date_DEdition,Date_limite
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,entreprise spécialisée dans l’ingénierie mécan...,gestionnaire des ressources humaines,gestionnaire des ressources humaines entrepris...,,,,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,description de la mission participation à la r...,stagiaire réseau fibre optique et informatique,stagiaire réseau fibre optique et informatique...,wandoo technologies,,stage – temps partiel,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,sous la responsabilité du responsable techniqu...,chiffreur btp,chiffreur btp sous la responsabilité du respon...,exceliam,master 2 ingéniorat bac 5,cdi,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,sous la responsabilité du responsable techniqu...,chargé daffaires senior,chargé daffaires senior sous la responsabilité...,exceliam,licence lmd bac 3 master 1 licence bac 4,cdi,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,dans le cadre de notre participation à un salo...,assistante anglais et français pour un salon d...,assistante anglais et français pour un salon d...,futuretech,,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,ASSISTANT(E) SOCIAL (E) PRESTATAIRE,NaN,centre...,06/05/2024,4,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117170-cou...,Abidjan,30/08/2024,...,hsd human resource solutionsrecrutecourtier en...,assistante social e prestataire,assistante social e prestataire hsd human reso...,,bac3,emploi,Health Alliance Côte d'Ivoire (HA-CI)recruteAS...,112994,06/05/2024,13/05/2024
59,COURTIER EN ASSURANCE,NaN,Abidjan,06/05/2024,2,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117169-tel...,Abidjan,11/05/2024,...,pok servicesrecrutetele vendeur\n \n \ndescrip...,courtier en assurance,courtier en assurance pok servicesrecrutetele ...,,bac,stage,HSD HUMAN RESOURCE SOLUTIONSrecruteCOURTIER EN...,112993,06/05/2024,30/08/2024
60,TELE VENDEUR,NaN,Abidjan,06/05/2024,3,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117168-com...,"Abidjan (Yopougon), Côte dIvoire",15/05/2024,...,domafrikrecrutecomptable\n \n \ndescription du...,tele vendeur,tele vendeur domafrikrecrutecomptable\n \n \nd...,,bac3 bac4,emploi,POK ServicesrecruteTELE VENDEURDescription du ...,112992,06/05/2024,11/05/2024
61,COMPTABLE,NaN,(...,06/05/2024,1,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117167-bar...,Abidjan,20/05/2024,...,potentiel innov rhrecrutebarman hf\n \n \ndesc...,comptable,comptable potentiel innov rhrecrutebarman hf\n...,,bac,emploi,DOMAFRIKrecruteCOMPTABLEDescription du poste

In [23]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import os
import dateparser

def alerteemploi():
    def collect_job_info(url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                job_title = soup.find('h3', class_='title').text.strip()
                author = soup.find('div', class_='info').find('li').text.strip()
                date_posted = soup.find('div', class_='info').find_all('li')[1].text.strip()
                description = soup.find('div', class_='content').p.text.strip()
                post_url = url
                offre_links = [offer.find('a')['href'] for offer in soup.find_all('h3', class_='title')]
                job_data_list = []
                for offre_link in offre_links:
                    job_data = {
                        'Job Title': job_title,
                        'Author': author,
                        'Date Posted': date_posted,
                        'Description': description,
                        'URL': post_url,
                        'Post URL': offre_link
                    }
                    job_data_list.append(job_data)
                return job_data_list
            else:
                print("Failed to fetch the page.")
                return []
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return []

    urls = ["https://alerteemploi.net/category/emploi-stages/page/{}/".format(category) for category in range(52)]
    job_data_list = []
    for url in urls:
        job_info = collect_job_info(url)
        if job_info:
            job_data_list.extend(job_info)

    df = pd.DataFrame(job_data_list)

    def extract_job_specific_info(job_info):
        if job_info is None:
            return {}
        info_lines = job_info.split('\n')
        job_specific_info = {}
        for line in info_lines:
            if ':' in line:
                key, value = line.split(':', 1)
                job_specific_info[key.strip()] = value.strip()
        return job_specific_info

    def collect_job_info(urls):
        job_info_list = []
        for url in urls:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    job_title_element = soup.find('h1', class_='entry-title')
                    job_title = job_title_element.text.strip() if job_title_element else None
                    author_container = soup.find('div', class_='info')
                    author_element = author_container.find_all('li')[1] if author_container else None
                    author = author_element.text.strip() if author_element else None
                    date_container = soup.find('div', class_='info')
                    date_element = date_container.find_all('li')[0] if date_container else None
                    date_posted = date_element.text.strip() if date_element else None
                    description_container = soup.find('div', class_='sc-1fwbcuw-0')
                    description = description_container.text.strip() if description_container else None
                    job_data = {
                        'Job Title1': job_title,
                        'Author1': author,
                        'Date Posted1': date_posted,
                        'Description1': description,
                        'Post URL': url
                    }
                    job_info_list.append(job_data)
                else:
                    print(f"Failed to fetch the page at URL: {url}")
                    job_data = {
                        'Post URL': url
                    }
                    job_info_list.append(job_data)
            except Exception as e:
                print(f"An error occurred while processing URL {url}: {str(e)}")
        return job_info_list

    job_listing_urls = list(df['Post URL'])
    job_info_list = collect_job_info(job_listing_urls)

    if job_info_list:
        df_init = pd.DataFrame(job_info_list)
        job_specific_info = [extract_job_specific_info(job['Description1']) for job in job_info_list]
        df1 = pd.DataFrame(job_specific_info)
        concatenated_df = pd.concat([df_init, df1], axis=1)
        concatenated_df.drop_duplicates()
        data_df=pd.merge(df, concatenated_df, on='Post URL')
        return data_df.drop_duplicates()
    else:
        print("Aucune information sur les offres d'emploi n'a été trouvée.")
        return None

result = alerteemploi()
result



,Job Title,Author,Date Posted,Description,URL,Post URL,Job Title1,Author1,Date Posted1,Description1,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat
0,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
2,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
4,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,EMPOWER TALENTS & CAREERS recrute RESPONSABLE...,admin,"April 28, 2018",EMPOWER TALENTS & CAREERS recrute RESPONSABLE ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/office-pro-community-...,None,admin,"April 23, 2018",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,EMPOWER TALENTS & CAREERS recrute RESPONSABLE...,admin,"April 28, 2018",EMPOWER TALENTS & CAREERS recrute RESPONSABLE ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/prestimex-ci-ingenieu...,None,admin,"April 22, 2018",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,EMPOWER TALENTS & CAREERS recrute RESPONSABLE...,admin,"April 28, 2018",EMPOWER TALENTS & CAREERS recrute RESPONSABLE ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/institut-francais-de-...,None,admin,"April 22, 2018",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,EMPOWER TALENTS & CAREERS recrute RESPONSABLE...,admin,"April 28, 2018",EMPOWER TALENTS & CAREERS recrute RESPONSABLE ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/intranlog-commercial-...,None,admin,"April 20, 2018",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
result

,Job Title,Author,Date Posted,Description,URL,Post URL,Job Title1,Author1,Date Posted1,Description1,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat
0,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
1,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
2,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/trouver-un-emploi-san...,None,Alerte Emploi,"August 2, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/trouver-un-emploi-san...,None,Alerte Emploi,"August 2, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/concours-de-gendarmer...,None,Alerte Emploi,"July 27, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/concours-de-gendarmer...,None,Alerte Emploi,"July 27, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import os
import dateparser

def alerteemploi():
    def collect_job_info(url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                job_title_element = soup.find('h3', class_='title')
                job_title = job_title_element.text.strip() if job_title_element else None
                author_element = soup.find('div', class_='info').find('li')
                author = author_element.text.strip() if author_element else None
                date_posted_element = soup.find('div', class_='info').find_all('li')[1]
                date_posted = date_posted_element.text.strip() if date_posted_element else None
                description_element = soup.find('div', class_='content').p
                description = description_element.text.strip() if description_element else None
                post_url = url
                offer_links = [offer.find('a')['href'] for offer in soup.find_all('h3', class_='title')]
                job_data_list = []
                for offer_link in offer_links:
                    job_data = {
                        'Job Title': job_title,
                        'Author': author,
                        'Date Posted': date_posted,
                        'Description': description,
                        'URL': post_url,
                        'Post URL': offer_link
                    }
                    job_data_list.append(job_data)
                return job_data_list
            else:
                print("Failed to fetch the page.")
                return []
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return []

    urls = ["https://alerteemploi.net/category/emploi-stages/page/{}/".format(category) for category in range(52)]
    job_data_list = []
    for url in urls:
        job_info = collect_job_info(url)
        if job_info:
            job_data_list.extend(job_info)

    df = pd.DataFrame(job_data_list)

    def extract_job_specific_info(job_info):
        if job_info is None:
            return {}
        info_lines = job_info.split('\n')
        job_specific_info = {}
        for line in info_lines:
            if ':' in line:
                key, value = line.split(':', 1)
                job_specific_info[key.strip()] = value.strip()
        return job_specific_info

    def collect_job_info(urls):
        job_info_list = []
        for url in urls:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    job_title_element = soup.find('h1', class_='entry-title')
                    job_title = job_title_element.text.strip() if job_title_element else None
                    author_container = soup.find('div', class_='info')
                    author_element = author_container.find_all('li')[1] if author_container else None
                    author = author_element.text.strip() if author_element else None
                    date_container = soup.find('div', class_='info')
                    date_element = date_container.find_all('li')[0] if date_container else None
                    date_posted = date_element.text.strip() if date_element else None
                    description_container = soup.find('div', class_='sc-1fwbcuw-0')
                    description = description_container.text.strip() if description_container else None
                    job_data = {
                        'Job Title1': job_title,
                        'Author1': author,
                        'Date Posted1': date_posted,
                        'Description1': description,
                        'Post URL': url
                    }
                    job_info_list.append(job_data)
                else:
                    print(f"Failed to fetch the page at URL: {url}")
                    job_data = {
                        'Post URL': url
                    }
                    job_info_list.append(job_data)
            except Exception as e:
                print(f"An error occurred while processing URL {url}: {str(e)}")
        return job_info_list

    job_listing_urls = list(df['Post URL'])
    job_info_list = collect_job_info(job_listing_urls)

    if job_info_list:
        df_init = pd.DataFrame(job_info_list)
        job_specific_info = [extract_job_specific_info(job['Description1']) for job in job_info_list]
        df1 = pd.DataFrame(job_specific_info)
        concatenated_df = pd.concat([df_init, df1], axis=1)
        concatenated_df.drop_duplicates(inplace=True)
        data_df = pd.merge(df, concatenated_df, on='Post URL')
        data_df.drop_duplicates(inplace=True)
        return data_df
    else:
        print("Aucune information sur les offres d'emploi n'a été trouvée.")
        return None

result = alerteemploi()
result


,Job Title,Author,Date Posted,Description,URL,Post URL,Job Title1,Author1,Date Posted1,Description1,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat
0,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
1,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
2,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/trouver-un-emploi-san...,None,Alerte Emploi,"August 2, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/trouver-un-emploi-san...,None,Alerte Emploi,"August 2, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/concours-de-gendarmer...,None,Alerte Emploi,"July 27, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/concours-de-gendarmer...,None,Alerte Emploi,"July 27, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
equivalences = {
    'Job Title': 'INTITULE_DU_POSTE',
'Date Posted': 'DATE_DE_DEBUT_DE_L_OFFRE',
'Description': 'Description',
'URL': 'URL',
'Post URL': 'Post URL',
    }
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
def renommer_colonnes(df, equivalences):
    colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
    df_renomme = df.rename(columns=colonnes_renommees)
    return df_renomme
    
    
    # Renommer les colonnes du DataFrame
result1 = renommer_colonnes(result, equivalences)
result1

,INTITULE_DU_POSTE,Author,DATE_DE_DEBUT_DE_L_OFFRE,Description,URL,Post URL,Job Title1,Author1,Date Posted1,Description1,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat
0,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
1,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-chez-msf-...,None,Alerte Emploi,"February 5, 2024","Médecins Sans Frontières fondée en 1971, est u...",RESPONSABLE RESSOURCES HUMAINES OPERATIONNELLES,Ressources Humaines,Directeur RH,"Spécialiste RH, Référent RH Urgences, Assistan...",100%,Abidjan,Toutes les Zones d’intervention MSF – WaCA,CDD Renouvelable,Le plus tôt possible.
2,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/air-ci-lance-son-recr...,None,Alerte Emploi,"February 4, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/recrutement-unicef-2024/,None,Alerte Emploi,"February 2, 2024",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/trouver-un-emploi-san...,None,Alerte Emploi,"August 2, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/trouver-un-emploi-san...,None,Alerte Emploi,"August 2, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/concours-de-gendarmer...,None,Alerte Emploi,"July 27, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Recrutement chez MSF 2024,Alerte Emploi,"February 5, 2024",Description de l’offre d’emploi Médecins Sans ...,https://alerteemploi.net/category/emploi-stage...,https://alerteemploi.net/concours-de-gendarmer...,None,Alerte Emploi,"July 27, 2023",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df_final = pd.concat([df_final,result1 ], axis=0, ignore_index=True)
df_final


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Description1,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Recrutement chez MSF 2024,NaN,NaN,"February 5, 2024",NaN,NaN,https://alerteemploi.net/category/emploi-stage...,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,Recrutement chez MSF 2024,NaN,NaN,"February 5, 2024",NaN,NaN,https://alerteemploi.net/category/emploi-stage...,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,Recrutement chez MSF 2024,NaN,NaN,"February 5, 2024",NaN,NaN,https://alerteemploi.net/category/emploi-stage...,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Recrutement chez MSF 2024,NaN,NaN,"February 5, 2024",NaN,NaN,https://alerteemploi.net/category/emploi-stage...,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Description1,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,COURTIER EN ASSURANCE,NaN,Abidjan,06/05/2024,2,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117169-tel...,Abidjan,11/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,TELE VENDEUR,NaN,Abidjan,06/05/2024,3,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117168-com...,"Abidjan (Yopougon), Côte dIvoire",15/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,COMPTABLE,NaN,(...,06/05/2024,1,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117167-bar...,Abidjan,20/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,Barman (H/F),NaN,Abidjan,06/05/2024,5,NaN,https://emploi.educarriere.ci/nos-offres?page1...,https://emploi.educarriere.ci/offre-117183-com...,Côte d'Ivoire,18/05/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
result1['Post URL'][50]

'https://alerteemploi.net/bourse-hdr-de-luniversite-bond-2023-24-en-australie-entierement-financee/'

In [ ]:
"https://alerteemploi.net/la-sgbci-recrute-responsable-risques-operationnels-et-supervision-manageriale/"

In [89]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Liste des URLs des pages à extraire
urls = [
   "https://alerteemploi.net/la-sgbci-recrute-responsable-risques-operationnels-et-supervision-manageriale/",
   #"https://alerteemploi.net/newcrest-mining-senior-social-performance-officer/"
]

# Initialiser une liste pour stocker les DataFrames de chaque page
dfs = []

# Parcourir chaque URL dans la liste
for url in urls:
    # Faire une requête GET à l'URL
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Analyser le contenu HTML de la page avec BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver la balise div avec la classe "single-post-content-wrap" qui contient toutes les informations
        content_div = soup.find('div', class_='single-post-content-wrap')

        # Extraire les informations à partir des balises ul avec la classe "text-small criterias-show"
        informations = {}
        ul_tags = content_div.find_all('ul', class_='text-small criterias-show')
        for ul_tag in ul_tags:
            li_tags = ul_tag.find_all('li')
            for li_tag in li_tags:
                span_tags = li_tag.find_all('span')
                if len(span_tags) == 2:
                    key = span_tags[0].text.strip().replace(":", "")
                    value = span_tags[1].text.strip()
                    informations[key] = value

        # Ajouter les autres informations de la description
        other_informations = {}
        paragraphs = content_div.find_all('p')
        for paragraph in paragraphs:
            strong_tag = paragraph.find('strong')
            if strong_tag:
                key = strong_tag.text.strip().replace(":", "")
                value = paragraph.text.strip().replace(strong_tag.text.strip(), "")
                other_informations[key] = value

        # Combiner les informations extraites
        all_informations = {**informations, **other_informations}

        # Créer un DataFrame à partir du dictionnaire
        df = pd.DataFrame([all_informations])

        # Ajouter le DataFrame à la liste
        dfs.append(df)

    else:
        print("Erreur lors de la récupération de la page :", response.status_code)

# Concaténer tous les DataFrames de la liste en un seul DataFrame
result_df1 = pd.concat(dfs, ignore_index=True)

# Afficher le DataFrame résultant
result_df1


,Secteur d’activité/,Lieu de travail/,Date d’expiration/,Nombre de postes/,Niveau de poste/,Niveau d’étude (diplome)/,Type de contrat/,Environnement de travail,Mission du poste,Profil,Formation,Expérience,Savoir,Savoir faire,Savoir être,Candidater,e 1er mai 2018,NB Seuls les candidats sélectionnés seront contactés.,
0,"Banque, Assurance, Finance",Côte d’ivoire,16 Juin,01 poste ouvert,Confirmé / Expérimenté,"Master 1, Licence Bac + 4| Master 2, Ingéniora...",CDI,,,,,,,,,,"Pour faire acte de candidature, veuillez envoy...",,


In [90]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract_information_from_url(url):
    # Faire une requête GET à l'URL
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Analyser le contenu HTML de la page avec BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver la balise div avec la classe "single-post-content-wrap" qui contient toutes les informations
        content_div = soup.find('div', class_='single-post-content-wrap')

        # Initialiser un dictionnaire pour stocker les informations
        informations_dict = {}

        # Extraire les informations à partir des balises ul avec la classe "list-group" si elles existent
        ul_tag = content_div.find('ul', class_='list-group')
        if ul_tag:
            li_tags = ul_tag.find_all('li')
            for li_tag in li_tags:
                strong_tag = li_tag.find('strong')
                if strong_tag:
                    key = strong_tag.text.strip().replace(":", "")
                    value = li_tag.text.strip().replace(strong_tag.text.strip(), "")
                    informations_dict[key] = value

        # Ajouter les autres informations de la description
        paragraphs = content_div.find_all('p')
        for paragraph in paragraphs:
            strong_tag = paragraph.find('strong')
            if strong_tag:
                key = strong_tag.text.strip().replace(":", "")
                value = paragraph.text.strip().replace(strong_tag.text.strip(), "")
                informations_dict[key] = value

        return informations_dict

    else:
        print("Erreur lors de la récupération de la page :", response.status_code)
        return None

# Liste des URLs des pages à extraire
urls = [
    "https://alerteemploi.net/newcrest-mining-senior-social-performance-officer/",
    #"https://alerteemploi.net/la-sgbci-recrute-responsable-risques-operationnels-et-supervision-manageriale/",
    # Ajoutez d'autres URLs ici si nécessaire
]

# Initialiser une liste pour stocker les DataFrames de chaque page
dfs = []

# Parcourir chaque URL dans la liste
for url in urls:
    # Extraire les informations de l'URL
    informations_dict = extract_information_from_url(url)

    # Si des informations ont été extraites, créer un DataFrame et l'ajouter à la liste
    if informations_dict:
        df = pd.DataFrame([informations_dict])
        dfs.append(df)

# Concaténer tous les DataFrames de la liste en un seul DataFrame
result_df = pd.concat(dfs, ignore_index=True)

# Afficher le DataFrame résultant
result_df


,Poste,Type d’offre,Métier(s),Niveau(x),Expérience,Lieu,NEWCREST MINING,Aperçu,Date de clôture 05 Mai 2018
0,SENIOR SOCIAL PERFORMANCE OFFICER,Emploi (CDI),"Communication, Sciences sociales",BAC+4,3 ans,Bonikro,\nrecrute\nSENIOR SOCIAL PERFORMANCE OFFICER,\nNous recherchons actuellement un Senior Soci...,


In [ ]:

Secteur d’activité/	Lieu de travail/	Date d’expiration/	Nombre de postes/	Niveau de poste/	Niveau d’étude (diplome)/	Type de contrat/	Environnement de travail	Mission du poste	Profil	Formation	Expérience	Savoir	Savoir faire	Savoir être	Candidater	e 1er mai 2018	NB  Seuls les candidats sélectionnés seront contactés.	
0	Banque, Assurance, Finance	Côte d’ivoire	16 Juin	01 poste ouvert	Confirmé / Expérimenté	Master 1, Licence Bac + 4| Master 2, Ingéniora...	CDI										Pour faire acte de candidature, veuillez envoy...		
et Poste	Type d’offre	Métier(s)	Niveau(x)	Expérience	Lieu	NEWCREST MINING	Aperçu	Date de clôture 05 Mai 2018
0	SENIOR SOCIAL PERFORMANCE OFFICER	Emploi (CDI)	Communication, Sciences sociales	BAC+4	3 ans	Bonikro	\nrecrute\nSENIOR SOCIAL PERFORMANCE OFFICER	\nNous recherchons actuellement un Senior Soci...	

In [78]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def extract_information_from_url(url):
    # Faire une requête GET à l'URL
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Analyser le contenu HTML de la page avec BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver la balise div avec la classe "single-post-content-wrap" qui contient toutes les informations
        content_div = soup.find('div', class_='single-post-content-wrap')

        # Initialiser un dictionnaire pour stocker les informations
        informations_dict = {}

        # Extraire les informations à partir des balises ul avec la classe "list-group" si elles existent
        ul_tag = content_div.find('ul', class_='list-group')
        if ul_tag:
            li_tags = ul_tag.find_all('li')
            for li_tag in li_tags:
                strong_tag = li_tag.find('strong')
                if strong_tag:
                    key = strong_tag.text.strip().replace(":", "")
                    value = li_tag.text.strip().replace(strong_tag.text.strip(), "")
                    informations_dict[key] = value

        # Ajouter les autres informations de la description
        paragraphs = content_div.find_all('p')
        for paragraph in paragraphs:
            strong_tag = paragraph.find('strong')
            if strong_tag:
                key = strong_tag.text.strip().replace(":", "")
                value = paragraph.text.strip().replace(strong_tag.text.strip(), "")
                informations_dict[key] = value

        return informations_dict

    else:
        print("Erreur lors de la récupération de la page :", response.status_code)
        return None

# Liste des URLs des pages à extraire
urls = [
    "https://alerteemploi.net/newcrest-mining-senior-social-performance-officer/",
    "https://alerteemploi.net/la-sgbci-recrute-responsable-risques-operationnels-et-supervision-manageriale/",
    # Ajoutez d'autres URLs ici si nécessaire
]

# Initialiser une liste pour stocker les DataFrames de chaque page
dfs = []

# Parcourir chaque URL dans la liste
for url in urls:
    # Extraire les informations de l'URL
    informations_dict = extract_information_from_url(url)

    # Si des informations ont été extraites, créer un DataFrame et l'ajouter à la liste
    if informations_dict:
        df = pd.DataFrame([informations_dict])
        dfs.append(df)

# Concaténer tous les DataFrames de la liste en un seul DataFrame
result_df = pd.concat(dfs, ignore_index=True)

# Ajouter une colonne spécifiant l'URL correspondant à chaque entrée
result_df['URL'] = urls

# Réorganiser les colonnes pour correspondre à l'ordre spécifié


# Afficher le DataFrame résultant
result_df


,Poste,Type d’offre,Métier(s),Niveau(x),Expérience,Lieu,NEWCREST MINING,Aperçu,Date de clôture 05 Mai 2018,Environnement de travail,...,Profil,Formation,Savoir,Savoir faire,Savoir être,Candidater,e 1er mai 2018,NB Seuls les candidats sélectionnés seront contactés.,,URL
0,SENIOR SOCIAL PERFORMANCE OFFICER,Emploi (CDI),"Communication, Sciences sociales",BAC+4,3 ans,Bonikro,\nrecrute\nSENIOR SOCIAL PERFORMANCE OFFICER,\nNous recherchons actuellement un Senior Soci...,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://alerteemploi.net/newcrest-mining-senio...
1,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,,...,,,,,,,"Pour faire acte de candidature, veuillez envoy...",,,https://alerteemploi.net/la-sgbci-recrute-resp...


In [101]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import os

from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import dateparser


def mondiale_ci():
    # Fonction pour extraire les données d'une page
    def scrape_page(url):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Lèvera une exception si la requête a échoué
            soup = BeautifulSoup(response.content, "html.parser")
            job_rows = soup.find("table", class_="results-table").find("tbody").find_all("tr")

            # Listes pour stocker les données de cette page
            job_titles = []
            locations = []
            job_families = []
            deadlines = []

            # Parcourir chaque ligne du tableau et extraire les informations nécessaires
            for row in job_rows:
                # Récupérer le titre de l'emploi
                job_title = row.find("a").text.strip()
                job_titles.append(job_title)

                # Récupérer l'emplacement
                location = row.find_all("td")[1].text.strip()
                locations.append(location)

                # Récupérer la famille d'emploi
                job_family = row.find_all("td")[2].text.strip()
                job_families.append(job_family)

                # Récupérer la date limite d'application
                deadline = row.find_all("td")[3].text.strip()
                deadlines.append(deadline)

            # Retourner les données de cette page sous forme de DataFrame
            data = {
                "Job Title": job_titles,
                "Location": locations,
                "Job Family": job_families,
                "Deadline": deadlines
            }
            return pd.DataFrame(data)
        except requests.exceptions.RequestException as e:
            print("Une erreur s'est produite lors de la requête:", e)
            return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur

    # URL de la première page
    base_url = "https://worldbankgroup.csod.com/ats/careersite/search.aspx?site=1&c=worldbankgroup&sid=%5e%5e%5eFLGscZMYY2RrwVaMR%2ftHYw%3d%3d"

    # Créer une liste pour stocker les DataFrames de chaque page
    dfs = []

    # Extraire les données de la première page
    dfs.append(scrape_page(base_url))

    # Trouver le nombre total de pages
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")
    pagination_span = soup.find("span", class_=re.compile(r"\btext\b"), text=re.compile(r"\d+"))
    if pagination_span:
        num_pages = int(pagination_span.text.strip())
    else:
        num_pages = 1

    # Boucle à travers chaque page et extraire les données
    for page_num in range(1, num_pages + 1):
        page_url = f"{base_url}&pg={page_num}"
        df = scrape_page(page_url)
        if not df.empty:  # Vérifier si le DataFrame n'est pas vide
            dfs.append(df)

    # Concaténer tous les DataFrames en un seul
    df_mondiale = pd.concat(dfs, ignore_index=True)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Job Title": "INTITULE_DU_POSTE",
        "Location": "LIEU_DU_POSTE_DE_TRAVAIL",
        "Job Family": "SECTEUR",
        "Deadline": "DATE_D_EXPIRATION_DE_L_OFFRE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_mondiale = renommer_colonnes(df_mondiale, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_mondiale

mondiale_ci=mondiale_ci()
mondiale_ci

,INTITULE_DU_POSTE,LIEU_DU_POSTE_DE_TRAVAIL,SECTEUR,DATE_D_EXPIRATION_DE_L_OFFRE
0,(Associate) Portfolio Officer,"Washington, DC,United States",Investment,5/17/2024
1,Associate Investment Officer,"Mexico City,Mexico More",Investment,4/15/2024
2,Associate Investment Officer,"Lima,Peru",Investment,5/7/2024
3,Associate Investment Officer,"Lagos,Nigeria",Investment,5/21/2024
4,Associate Investment Officer,"Santiago,Chile",Investment,5/20/2024
5,Associate Operations Officer,"Mexico City,Mexico",Operations,5/16/2024
6,Biodiversity Specialist,"Washington, DC,United States",Environment,5/24/2024
7,Business Analyst (ETC),"Chennai,India",Treasury,5/7/2024
8,Category Associate,"Washington, DC,United States",Procurement,5/16/2024
9,Climate Change Specialist,"Washington, DC,United States",Climate Change,5/17/2024


In [119]:
df_final = pd.concat([df_final,mondiale_ci], axis=0, ignore_index=True)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat,SECTEUR
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,E T Consultant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bogota,Colombia",5/10/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment
120,E T Consultant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC,United States",5/7/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investment
121,E T Consultant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC,United States",5/10/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Economics
122,E T Consultant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tbilisi,Georgia",5/14/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Energy


In [120]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Nous Recherchons,Département,Supérieur hiérarchique,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat,SECTEUR
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Climate Change Specialist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC,United States",5/17/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Climate Change
70,"Communications Associate - social media, GPE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC,United States",5/15/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Communications
71,Communications Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Istanbul,Turkiye",5/7/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Communications
72,"Counsel – Data, Technology and Cyber Risk",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Washington, DC,United States",5/13/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Legal


In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import os
#Importation du module d'envoi de mail
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import dateparser


def projobivoire():
    def extract_text(element, tag_name=None):
        if element and tag_name:
            tag = element.find(tag_name)
            return tag.text.strip() if tag else ""
        return ""

    def clean_text(text):
        return text.replace('\r\n', '').replace('\xa0', '')

    def scrape_projobivoire_page(page_url):
        job_data_list = []

        for url in page_url:
            try:
                response = requests.get(url, timeout=500)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                    print(f"Erreur de connexion à {url} : {e}")
                    continue

            soup = BeautifulSoup(response.text, 'html.parser')

            job_items = soup.find_all('div', class_='loop-item-wrap list')

            if not job_items:
                print(f"Aucun élément de travail trouvé pour l'URL : {url}")
                continue

            for job_item in job_items:
                title_tag = job_item.find('h3', class_='loop-item-title')
                title = extract_text(title_tag, 'a')

                job_type_tag = job_item.find('span', class_='job-type')
                job_type = extract_text(job_type_tag, 'span')

                job_date_posted = soup.find('span', class_='job-date__posted').text.strip()

                job_date_closing_tag = soup.find('span', class_='job-date__closing')
                job_date_closing = job_date_closing_tag.text.strip() if job_date_closing_tag else ""

                job_date_closing = job_date_closing.lstrip('-').strip()

                category_tag = job_item.find('span', class_='job-category')
                category = extract_text(category_tag, 'a')

                # Ajout de ces lignes pour extraire l'URL de l'e-mail
                email_url_tag = job_item.find('span', class_='noo-tool-email-job')
                email_url = email_url_tag['data-url'] if email_url_tag else ""

                data = {
                    'Title': title,
                    'Type': job_type,
                    'DatePosted': job_date_posted,
                    'DateClosing': job_date_closing,
                    'Category': category,
                    'EmailURL': email_url,
                    'URL': url
                }

                job_data_list.append(data)

        return job_data_list

    # Liste des URL de pages avec plusieurs offres d'emploi
    page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(546)]

    # Scrape des détails de chaque offre d'emploi sur les pages
    job_data_list = scrape_projobivoire_page(page_urls)

    # Création d'un DataFrame à partir de la liste des données d'emploi
    df_projobivoire = pd.DataFrame(job_data_list)

    # Affichage du DataFrame

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Title": "INTITULE_DU_POSTE",
        "Type": "TYPE_DE_CONTRAT_DU_POSTE",
        "DatePosted": "DATE_DE_PUBLICATION",
        "DateClosing": "DATE_D_EXPIRATION_DE_L_OFFRE",
        "Category": "CATEGORIE",
        "EmailURL": None,
        "URL": "SITE_WEB_DE_L_ENTREPRISE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_projobivoire = renommer_colonnes(df_projobivoire, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_projobivoire

projobivoire=projobivoire()
projobivoire

,INTITULE_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,DATE_DE_PUBLICATION,DATE_D_EXPIRATION_DE_L_OFFRE,CATEGORIE,EmailURL,SITE_WEB_DE_L_ENTREPRISE
0,Infirmier(ère),Appel d'offre,7 mai 2024,31 mai 2024,Médecine/Santé,https://projobivoire.com/jobs/infirmierere/,https://projobivoire.com/page/0/
1,RECRUTEMENT AGENT DE SECURITE PRIVEE DANS LA Z...,Stage,7 mai 2024,31 mai 2024,Sécurité,https://projobivoire.com/jobs/recrutement-agen...,https://projobivoire.com/page/0/
2,Recrutement RONDIER / CONTROLEUR,Emploi,7 mai 2024,31 mai 2024,Sécurité,https://projobivoire.com/jobs/recrutement-rond...,https://projobivoire.com/page/0/
3,Ingénieur Devops GCP,Emploi,7 mai 2024,31 mai 2024,Informatique,https://projobivoire.com/jobs/ingenieur-devops...,https://projobivoire.com/page/0/
4,Secrétaire informatique / communication,Contract,7 mai 2024,31 mai 2024,Assistanat de Direction,https://projobivoire.com/jobs/secretaire-infor...,https://projobivoire.com/page/0/
...,...,...,...,...,...,...,...
9277,Atlantique Microfinance recrute CHARGE(E) DE R...,Emploi,18 novembre 2021,1 décembre 2021,Finance,https://projobivoire.com/jobs/atlantique-micro...,https://projobivoire.com/page/545/
9278,SUCAF CI recrute ELECTRICIEN AUTOMOBILE ET CLI...,Emploi,18 novembre 2021,1 décembre 2021,Electrotechnique/Electricité,https://projobivoire.com/jobs/sucaf-ci-recrute...,https://projobivoire.com/page/545/
9279,SUCAF CI recrute ASSISTANT(E) COORDINATION REC...,Emploi,18 novembre 2021,1 décembre 2021,Agro-alimentaire,https://projobivoire.com/jobs/sucaf-ci-recrute...,https://projobivoire.com/page/545/
9280,A2I recrute CAISSIERES,Emploi,18 novembre 2021,1 décembre 2021,Caisse,https://projobivoire.com/jobs/a2i-recrute-cais...,https://projobivoire.com/page/545/


In [122]:
df_final = pd.concat([df_final,projobivoire], axis=0, ignore_index=True)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat,SECTEUR,DATE_DE_PUBLICATION,CATEGORIE,EmailURL
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,Atlantique Microfinance recrute CHARGE(E) DE R...,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Finance,https://projobivoire.com/jobs/atlantique-micro...
9352,SUCAF CI recrute ELECTRICIEN AUTOMOBILE ET CLI...,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Electrotechnique/Electricité,https://projobivoire.com/jobs/sucaf-ci-recrute...
9353,SUCAF CI recrute ASSISTANT(E) COORDINATION REC...,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Agro-alimentaire,https://projobivoire.com/jobs/sucaf-ci-recrute...
9354,A2I recrute CAISSIERES,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Caisse,https://projobivoire.com/jobs/a2i-recrute-cais...


In [123]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Supervise,Travail %,Poste Basé à,Mobilité,Contrat,Début de contrat,SECTEUR,DATE_DE_PUBLICATION,CATEGORIE,EmailURL
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7649,GRH ACCESS recrute RESPONSABLE DES OPERATIONS,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Hôtellerie,https://projobivoire.com/jobs/grh-access-recru...
7650,ROXGOLD RECRUTE CAMP COORDINATOR,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Gestion,https://projobivoire.com/jobs/roxgold-recrute-...
7651,SUCAF CI recrute ELECTRICIEN AUTOMOBILE ET CLI...,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Electrotechnique/Electricité,https://projobivoire.com/jobs/sucaf-ci-recrute...
7652,SUCAF CI recrute ASSISTANT(E) COORDINATION REC...,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/545/,NaN,1 décembre 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18 novembre 2021,Agro-alimentaire,https://projobivoire.com/jobs/sucaf-ci-recrute...


In [106]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import os
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import dateparser

def emploi_ci():
    def extract_text(element, tag_name=None):
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""

    def clean_text(text):
        return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

        data_list = []

        for wrapper in job_description_wrappers:
            h5_tag = wrapper.find('h5')
            poste = extract_text(h5_tag, 'a')

            job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
            date_and_company = job_recruiter_tag.text.strip().split('|')
            date = date_and_company[0].strip() if date_and_company else ""
            entreprise = extract_text(job_recruiter_tag, 'a')

            description_tag = wrapper.find('div', class_='search-description')
            description = clean_text(description_tag.text.strip()) if description_tag else ""

            region_tag = wrapper.find('p', text='Région de :')
            region = extract_text(region_tag) if region_tag else ""

            data_list.append({
                'Poste': poste,
                'Entreprise': entreprise,
                'Date': date,
                'Description': description,
                'Région': region,
                'URL' : url
            })

        df = pd.DataFrame(data_list)
        return df

    # Liste des liens
    categories = ["31", "1127"]#, "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
    #categories=["31"]
    # Liste d'URLs générées
    urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

    # Créer un DataFrame à partir des liens
    df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)


    from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

    # Liste des liens

    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in list(df["URL"]):
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_="job-description-wrapper")

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h5')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link, 'URL' :url})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        # Afficher le DataFrame
        #print(all_job_details_df)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    emploi_df = pd.merge(df, all_job_details_df, on='URL')
    emploi_df = emploi_df.drop_duplicates()
    
    #df["URL"]=list(all_job_details_df["URL"])
    #df["Offre_Link"]=list(all_job_details_df["Offre_Link"])

    # Fonction pour extraire les informations d'une page
    def extract_information(url):
        try:
            response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
            response.raise_for_status()
            response.encoding = 'utf-8'

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraction des informations sur l'entreprise
            company_info = soup.select_one('.job-ad-company')
            entreprise = {
                "Offre_Link" : url,
                'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
                'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
                'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
            }

            # Extraction des informations sur l'annonce
            annonce_info = soup.select_one('.job-ad-details')
            annonce = {
                'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
                'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
                'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
                'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
                'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
                'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
                'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
                'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
                'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
                'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
                'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
                'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
            }

            return {'entreprise': entreprise, 'annonce': annonce}

        except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
            print(f"Erreur lors de la requête {url}: {e}")
            # Relancer la requête
            entreprise = {
                "Offre_Link" : url,
                'Nom': "",
                'Secteur d´activité': "",
                'Description de l\'entreprise':""}
            annonce = {'Poste':"",
                       'Missions': "",
                       'Profil recherché':"",
                       'Métier':"",
                       'Secteur d´activité (de l\'annonce)':"",
                       'Type de contrat':"",
                       'Région': "",
                       'Ville':"",
                       'Niveau d\'expérience':"",
                       'Niveau d\'études':"",
                       'Compétences clés':"",
                       'Nombre de poste(s)':""}


            return {'entreprise': entreprise, 'annonce': annonce}

    # Liste des URLs
    urls = list(emploi_df['Offre_Link'])

    # Initialisation d'une liste pour stocker les DataFrames
    df_list = []

    # Boucle à travers chaque URL
    for url in urls:
        data = extract_information(url)

        # Si la requête a échoué, passez à l'URL suivante
        if data is None:
            continue

        # Création du DataFrame pour chaque URL
        df = pd.DataFrame([data['entreprise'] | data['annonce']])

        # Ajout du DataFrame à la liste
        df_list.append(df)

    # Concaténation des DataFrames de chaque URL
    result_df = pd.concat(df_list, ignore_index=True)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    result_df = pd.merge(result_df, emploi_df, on='Offre_Link')
    result_df = emploi_df.drop_duplicates()


# Réorganiser les colonnes selon vos besoins
#Poste 	Entreprise 	Date 	Description 	Région 	URL 	Offre_Link

    return result_df

emploi_ci=emploi_ci()
emploi_ci


C:\Users\Dell\anaconda3\lib\site-packages\soupsieve\css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(


Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/java-developer-mf-1386343: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/java-jee-developer-mf-1386344: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/java-jee-developer-mf-1386344 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCB5B5FF0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/javascript-developer-mf-1386345: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/javascript-developer-mf-1386345 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCB5B5BD0>, 'Connection 

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/angular-developer-mf-1386339: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/angular-developer-mf-1386339 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCAC49180>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/devops-engineer-mf-1386340: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/devops-engineer-mf-1386340 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCA74DB10>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/dotnet-developer-mf-1386341: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-e

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/chef-projet-technique-transformation-reseaux-1411093: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/chef-projet-technique-transformation-reseaux-1411093 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCB15E740>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/technicien-support-ops-hf-1407949: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/technicien-support-ops-hf-1407949 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCB5B7BE0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/developpeur-front-end-1407411: HTTPSConnectionPool(host='ww

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/lead-developpeur-java-jee-hf-1417104: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/lead-developpeur-java-jee-hf-1417104 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCB15FF70>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCB15EA70>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/community-manager-creation-1418941: HTTPSConnectionPool(host='www.emploi.ci', port=

...

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/lead-developpeur-java-jee-hf-1417104: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/lead-developpeur-java-jee-hf-1417104 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCA788430>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCCE2E290>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/community-manager-creation-1418941: HTTPSConnectionPool(host='www.emploi.ci', port=

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-sap-abap-hf-1417099: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-sap-abap-hf-1417099 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCA788460>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-sap-fi-cops-hf-1417100: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-sap-fi-cops-hf-1417100 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCCE2E5C0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-sap-ehs-hf-1417101: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceede

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/java-jee-developer-mf-1386344: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/java-jee-developer-mf-1386344 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCCD5EC20>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/javascript-developer-mf-1386345: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/javascript-developer-mf-1386345 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCCD5FA30>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/kotlin-developer-mf-1386346: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with u

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/angular-developer-mf-1386339: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/angular-developer-mf-1386339 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCBEDE710>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/devops-engineer-mf-1386340: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/devops-engineer-mf-1386340 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CCCD5ED70>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/dotnet-developer-mf-1386341: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-e

,Poste,Entreprise,Date,Description,Région,URL,Offre_Link
0,Consultant SAP ABAP (H/F),AFRICASHORE,07.05.2024,Nous recherchons un Consultant SAP ABAP (H/F) ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
1,Consultant SAP ABAP (H/F),AFRICASHORE,07.05.2024,Nous recherchons un Consultant SAP ABAP (H/F) ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
2,Consultant SAP ABAP (H/F),AFRICASHORE,07.05.2024,Nous recherchons un Consultant SAP ABAP (H/F) ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
3,Consultant SAP ABAP (H/F),AFRICASHORE,07.05.2024,Nous recherchons un Consultant SAP ABAP (H/F) ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
4,Consultant SAP ABAP (H/F),AFRICASHORE,07.05.2024,Nous recherchons un Consultant SAP ABAP (H/F) ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
...,...,...,...,...,...,...,...
15020,NODE.JS Developer (M/F),AFRICASHORE,13.02.2024,We are looking for NODE.JS Developer profiles ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
15021,NODE.JS Developer (M/F),AFRICASHORE,13.02.2024,We are looking for NODE.JS Developer profiles ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
15022,NODE.JS Developer (M/F),AFRICASHORE,13.02.2024,We are looking for NODE.JS Developer profiles ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
15023,NODE.JS Developer (M/F),AFRICASHORE,13.02.2024,We are looking for NODE.JS Developer profiles ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...


In [124]:
df_final = pd.concat([df_final,emploi_ci], axis=0, ignore_index=True)
df_final
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Contrat,Début de contrat,SECTEUR,DATE_DE_PUBLICATION,CATEGORIE,EmailURL,Poste,Date,Région,Offre_Link
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7650,NaN,AFRICASHORE,NaN,NaN,NaN,NaN,https://www.emploi.ci/recherche-jobs-cote-ivoi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,IOS Developer (M/F),13.02.2024,,https://www.emploi.ci/offre-emploi-cote-ivoire...
7651,NaN,AFRICASHORE,NaN,NaN,NaN,NaN,https://www.emploi.ci/recherche-jobs-cote-ivoi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,JAVASCRIPT Developer (M/F),13.02.2024,,https://www.emploi.ci/offre-emploi-cote-ivoire...
7652,NaN,AFRICASHORE,NaN,NaN,NaN,NaN,https://www.emploi.ci/recherche-jobs-cote-ivoi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,KOTLIN Developer (M/F),13.02.2024,,https://www.emploi.ci/offre-emploi-cote-ivoire...
7653,NaN,AFRICASHORE,NaN,NaN,NaN,NaN,https://www.emploi.ci/recherche-jobs-cote-ivoi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LARAVEL Developer (M/F),13.02.2024,,https://www.emploi.ci/offre-emploi-cote-ivoire...


In [127]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import os
#Importation du module d'envoi de mail
from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
import dateparser


def projobivoire():
    def extract_text(element, tag_name=None):
        if element and tag_name:
            tag = element.find(tag_name)
            return tag.text.strip() if tag else ""
        return ""

    def clean_text(text):
        return text.replace('\r\n', '').replace('\xa0', '')

    def scrape_projobivoire_page(page_url):
        job_data_list = []

        for url in page_url:
            try:
                response = requests.get(url, timeout=500)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                    print(f"Erreur de connexion à {url} : {e}")
                    continue

            soup = BeautifulSoup(response.text, 'html.parser')

            job_items = soup.find_all('div', class_='loop-item-wrap list')

            if not job_items:
                print(f"Aucun élément de travail trouvé pour l'URL : {url}")
                continue

            for job_item in job_items:
                title_tag = job_item.find('h3', class_='loop-item-title')
                title = extract_text(title_tag, 'a')

                job_type_tag = job_item.find('span', class_='job-type')
                job_type = extract_text(job_type_tag, 'span')

                job_date_posted = soup.find('span', class_='job-date__posted').text.strip()

                job_date_closing_tag = soup.find('span', class_='job-date__closing')
                job_date_closing = job_date_closing_tag.text.strip() if job_date_closing_tag else ""

                job_date_closing = job_date_closing.lstrip('-').strip()

                category_tag = job_item.find('span', class_='job-category')
                category = extract_text(category_tag, 'a')

                # Ajout de ces lignes pour extraire l'URL de l'e-mail
                email_url_tag = job_item.find('span', class_='noo-tool-email-job')
                email_url = email_url_tag['data-url'] if email_url_tag else ""

                data = {
                    'Title': title,
                    'Type': job_type,
                    'DatePosted': job_date_posted,
                    'DateClosing': job_date_closing,
                    'Category': category,
                    'EmailURL': email_url,
                    'URL': url
                }

                job_data_list.append(data)

        return job_data_list

    # Liste des URL de pages avec plusieurs offres d'emploi
    page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(6)]

    # Scrape des détails de chaque offre d'emploi sur les pages
    job_data_list = scrape_projobivoire_page(page_urls)

    # Création d'un DataFrame à partir de la liste des données d'emploi
    df_projobivoire = pd.DataFrame(job_data_list)

    # Affichage du DataFrame

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Title": "INTITULE_DU_POSTE",
        "Type": "TYPE_DE_CONTRAT_DU_POSTE",
        "DatePosted": "DATE_DE_PUBLICATION",
        "DateClosing": "DATE_D_EXPIRATION_DE_L_OFFRE",
        "Category": "CATEGORIE",
        "EmailURL": None,
        "URL": "SITE_WEB_DE_L_ENTREPRISE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_projobivoire = renommer_colonnes(df_projobivoire, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_projobivoire

projobivoire=projobivoire()
projobivoire

,INTITULE_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,DATE_DE_PUBLICATION,DATE_D_EXPIRATION_DE_L_OFFRE,CATEGORIE,EmailURL,SITE_WEB_DE_L_ENTREPRISE
0,VIVO ENERGY CÔTE D’IVOIRE recrute,Emploi,8 mai 2024,17 mai 2024,Génie Civil/Travaux publics,https://projobivoire.com/jobs/vivo-energy-cote...,https://projobivoire.com/page/0/
1,CAISSIER(E) POUR UN FAST FOOD,Emploi,8 mai 2024,17 mai 2024,Finances/Comptabilité,https://projobivoire.com/jobs/caissiere-pour-u...,https://projobivoire.com/page/0/
2,BAOBAB+ Côte-d’Ivoire recrute deux (02) Consei...,Stage,8 mai 2024,17 mai 2024,Commerce/Ventes,https://projobivoire.com/jobs/baobab-cote-divo...,https://projobivoire.com/page/0/
3,TECHNICIEN EN PRODUCTION ANIMALE CONFIRME,Contrat,8 mai 2024,17 mai 2024,Elevage,https://projobivoire.com/jobs/technicien-en-pr...,https://projobivoire.com/page/0/
4,Livreurs avec permis,Contrat,8 mai 2024,17 mai 2024,Livraison,https://projobivoire.com/jobs/livreurs-avec-pe...,https://projobivoire.com/page/0/
...,...,...,...,...,...,...,...
97,CROSS BORDER ABIDJAN,Contrat,3 mai 2024,24 mai 2024,Communication,https://projobivoire.com/jobs/cross-border-abi...,https://projobivoire.com/page/5/
98,AGENT SENSIBILISATEUR,Emploi,3 mai 2024,24 mai 2024,Commerce/Ventes,https://projobivoire.com/jobs/agent-sensibilis...,https://projobivoire.com/page/5/
99,CROSS BORDER,Contrat,3 mai 2024,24 mai 2024,Banque,https://projobivoire.com/jobs/cross-border-3/,https://projobivoire.com/page/5/
100,GROUPE CICBAD HOLDING recrute CHARGEE DE COMMU...,Emploi,3 mai 2024,24 mai 2024,Commerce/Ventes,https://projobivoire.com/jobs/groupe-cicbad-ho...,https://projobivoire.com/page/5/


In [128]:
df_final = pd.concat([df_final,projobivoire], axis=0, ignore_index=True)
df_final
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Contrat,Début de contrat,SECTEUR,DATE_DE_PUBLICATION,CATEGORIE,EmailURL,Poste,Date,Région,Offre_Link
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7648,CAISSIER(E) POUR UN FAST FOOD,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/0/,NaN,17 mai 2024,...,NaN,NaN,NaN,8 mai 2024,Finances/Comptabilité,https://projobivoire.com/jobs/caissiere-pour-u...,NaN,NaN,NaN,NaN
7649,BAOBAB+ Côte-d’Ivoire recrute deux (02) Consei...,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/0/,NaN,17 mai 2024,...,NaN,NaN,NaN,8 mai 2024,Commerce/Ventes,https://projobivoire.com/jobs/baobab-cote-divo...,NaN,NaN,NaN,NaN
7650,TECHNICIEN EN PRODUCTION ANIMALE CONFIRME,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/0/,NaN,17 mai 2024,...,NaN,NaN,NaN,8 mai 2024,Elevage,https://projobivoire.com/jobs/technicien-en-pr...,NaN,NaN,NaN,NaN
7651,Livreurs avec permis,NaN,NaN,NaN,NaN,NaN,NaN,https://projobivoire.com/page/0/,NaN,17 mai 2024,...,NaN,NaN,NaN,8 mai 2024,Livraison,https://projobivoire.com/jobs/livreurs-avec-pe...,NaN,NaN,NaN,NaN


In [130]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

def agence_emploi_jeunes():
    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Liste des URLs à scraper
    urls = ["https://agenceemploijeunes.ci/site/offres-emplois?page={}".format(category) for category in range(31)]

    # Listes pour stocker les données
    job_titles = []
    publication_dates = []
    application_deadlines = []
    locations = []
    job_descriptions = []
    job_types = []
    diploma_requirements = []
    url_lien = []

    # Loop à travers chaque URL
    for url in urls:
        # Envoyer une requête au site web
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        # Trouver les annonces d'emploi
        job_listings = soup.find_all('div', class_='post-bx')

        # Extract data from each job listing
        for job_listing in job_listings:
            url_lien.append(url)

            # Titre du poste
            job_title = job_listing.find('h4').text.strip()
            job_titles.append(job_title)

            # Dates de publication et de candidature
            date_info = job_listing.find_all('li', {'class': ''})
            if date_info:
                publication_date = date_info[0].text.replace('Publié le:', '').strip()
                application_deadline = date_info[1].text.replace('Date limite:', '').strip()
                publication_dates.append(publication_date)
                application_deadlines.append(application_deadline)

            # Localisation
            location = date_info[2].text.replace('ABENGOUROU', '').strip()
            locations.append(location)

            # Description du poste
            job_description = job_listing.find('p').text.strip()
            job_descriptions.append(job_description)

            # Type de poste
            job_type = job_listing.find('span', {'class': 'pull-right'}).text.strip()
            job_types.append(job_type)

            # Exigences en diplôme
            diploma_requirement = job_listing.find('div', {'class': 'salary-bx'}).text.strip()
            diploma_requirements.append(diploma_requirement)

    # Créer un DataFrame Pandas
    data = {
        'Job Title': job_titles,
        'Publication Date': publication_dates,
        'Application Deadline': application_deadlines,
        'Location': locations,
        'Job Description': job_descriptions,
        'Job Type': job_types,
        'Diploma Requirement': diploma_requirements,
        "URL": url_lien
    }

    df_agenceemploijeunes = pd.DataFrame(data)

    # Utiliser la méthode str.extract pour extraire la valeur après "Diplôme :"
    df_agenceemploijeunes['Diplome'] = df_agenceemploijeunes['Diploma Requirement'].str.extract(r'Diplôme :[ \t]([^\n\r])')

    # Scrapper les détails supplémentaires à partir des URL
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = 'C:\\Program Files (x86)\\chromedriver.exe'
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe" 
    #C:\Program Files\Google\Chrome\Application# Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # List to store job details
    all_job_details = []

    # Parcourir les liens
    for url in urls:
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)

        # Trouver toutes les offres d'emploi sur la page
        offer_links = soup.select('.post-bx h4 a')

        # Parcourir les liens d'offres
        for offer_link in offer_links:
            # Extraire l'URL de l'offre
            offer_url = offer_link.get('href')

            # Ajouter les détails à la liste
            all_job_details.append({'Offre_Link': offer_url, 'URL': url})

    # Fermer le navigateur à la fin
    driver.quit()

    # Créer un DataFrame avec les détails des offres d'emploi
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    df_agenceemploi_jeunes = pd.merge(df_agenceemploijeunes, all_job_details_df, on='URL')
    df_agenceemploi_jeunes = df_agenceemploi_jeunes.drop_duplicates()
    df_agenceemploi_jeunes.reset_index(drop=True, inplace=True)

    # Création d'un dictionnaire pour stocker les données
    job_data = {
        'Job Title': [],
        'Location': [],
        'Reference': [],
        'Number of Positions': [],
        'Closing Date': [],
        'Diploma': [],
        'Job Type': [],
        'Experience': [],
        'Education Level': [],
        'Gender': [],
        'Job Description': [],
        'Offre_Link': []
    }

    # Loop through each URL
    for url in df_agenceemploi_jeunes["Offre_Link"]:
        offre_url = url  # Sauvegarder l'URL même en cas d'exception
        try:
            # Send a request to the website
            req = requests.get(url, headers=headers)
            req.raise_for_status()  # Raise an error for unsuccessful responses
            soup = BeautifulSoup(req.text, 'html.parser')

            # Extract job details
            job_details = soup.find('div', class_='widget_getintuch')

            if job_details:
                # Extract data from job details
                ul_element = job_details.find('ul')
                if ul_element:
                    details_list = ul_element.find_all('li')

                    # Initialize variables to store details
                    location = reference = num_positions = closing_date = diploma = job_type = experience = education_level = gender = None

                    # Iterate through details
                    for detail in details_list:
                        label = detail.find('strong')
                        value_span = detail.find('span', class_='text-black-light')

                        if label and value_span:
                            label_text = label.text.strip()
                            value_text = value_span.text.strip()

                            if 'Lieu de travail' in label_text:
                                location = value_text
                            elif 'Reference' in label_text:
                                reference = value_text
                            elif 'Nombre de poste' in label_text:
                                num_positions = value_text
                            elif 'Date de clôture' in label_text:
                                closing_date = value_text
                            elif 'Diplôme' in label_text:
                                diploma = value_text
                            elif 'Type de contrat' in label_text:
                                job_type = value_text
                            elif 'Expérience professionnelle' in label_text:
                                experience = value_text
                            elif 'Niveau d\'études' in label_text:
                                education_level = value_text
                            elif 'Sexe' in label_text:
                                gender = value_text

                    # Append extracted details to the dictionary
                    job_data['Location'].append(location)
                    job_data['Reference'].append(reference)
                    job_data['Number of Positions'].append(num_positions)
                    job_data['Closing Date'].append(closing_date)
                    job_data['Diploma'].append(diploma)
                    job_data['Job Type'].append(job_type)
                    job_data['Experience'].append(experience)
                    job_data['Education Level'].append(education_level)
                    job_data['Gender'].append(gender)

                # Extract job title and description
                job_title_element = soup.find('h3', {'class': 'title-head'})
                if job_title_element:
                    job_title = job_title_element.text.strip()
                    job_data['Job Title'].append(job_title)

                    job_description_info = soup.find('div', {'class': 'job-info-box'}).find('ul')
                    if job_description_info:
                        job_description_text = '\n'.join([li.text.strip() for li in job_description_info.find_all('li')])
                        job_data['Job Description'].append(job_description_text)
                    else:
                        job_data['Job Description'].append(None)
                else:
                    job_data['Job Title'].append(None)
                    job_data['Job Description'].append(None)
            else:
                job_data['Job Title'].append(None)
                job_data['Job Description'].append(None)

            # Append URL to the dictionary
            job_data['Offre_Link'].append(offre_url)

        except requests.exceptions.RequestException as e:
            print(f"An error occurred while accessing URL: {url}")
            print(e)
            # Ajouter l'URL même en cas d'exception
            job_data['Location'].append(None)
            job_data['Reference'].append(None)
            job_data['Number of Positions'].append(None)
            job_data['Closing Date'].append(None)
            job_data['Diploma'].append(None)
            job_data['Job Type'].append(None)
            job_data['Experience'].append(None)
            job_data['Education Level'].append(None)
            job_data['Gender'].append(None)
            job_data['Job Title'].append(None)
            job_data['Job Description'].append(None)
            job_data['Offre_Link'].append(offre_url)

    # Create DataFrame from the collected data
    df_jobs = pd.DataFrame(job_data)
    agenceemploi_jeunes_df = pd.merge(df_agenceemploi_jeunes, df_jobs, on='Offre_Link')
    agenceemploi_jeunes_dfagenceemploi_jeunes_df = agenceemploi_jeunes_df.drop_duplicates()
    agenceemploi_jeunes_df.reset_index(drop=True, inplace=True)
    equivalences = {
    "Job Title_x": "INTITULE_DU_POSTE",
    "Publication Date": "DATE_DE_PUBLICATION",
    "Application Deadline": None,
    "Location_x": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Job Description_x": "DESCRIPTION_DU_POSTE",
    "Job Type_x": "TYPE_DE_CONTRAT_DU_POSTE",
    "Diploma Requirement": None,
    "URL": None,
    "Diplome": "DIPLOME_REQUIS",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    # Colonnes sans équivalence
    "Location_y": None,
    "Reference": None,
    "Number of Positions": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Closing Date": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Diploma": "DIPLOME",
    "Job Type_y": None,
    "Experience": None,
    "Education Level": "NIVEAU_D_ETUDES",
    "Gender": None,
    "Job Description_y": None}
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    agenceemploi_jeunes_df = renommer_colonnes(agenceemploi_jeunes_df, equivalences)
    
    return agenceemploi_jeunes_df
agenceemploi_jeunes_df=agence_emploi_jeunes()
agenceemploi_jeunes_df

An error occurred while accessing URL: https://agenceemploijeunes.ci/site/offres-emplois/35880
HTTPSConnectionPool(host='agenceemploijeunes.ci', port=443): Max retries exceeded with url: /site/offres-emplois/35880 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CD2C3BBE0>, 'Connection to agenceemploijeunes.ci timed out. (connect timeout=None)'))
An error occurred while accessing URL: https://agenceemploijeunes.ci/site/offres-emplois/35879
HTTPSConnectionPool(host='agenceemploijeunes.ci', port=443): Max retries exceeded with url: /site/offres-emplois/35879 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000027CD2BE0310>, 'Connection to agenceemploijeunes.ci timed out. (connect timeout=None)'))


,INTITULE_DU_POSTE,DATE_DE_PUBLICATION,Application Deadline,LIEU_DU_POSTE_DE_TRAVAIL,DESCRIPTION_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,Diploma Requirement,URL,DIPLOME_REQUIS,SITE_WEB_DE_L_ENTREPRISE,...,Location_y,Reference,NOMBRE_DE_POSTES_A_POURVOIR,DATE_D_EXPIRATION_DE_L_OFFRE,DIPLOME,Job Type_y,Experience,NIVEAU_D_ETUDES,Gender,Job Description_y
0,STAGIAIRES AGENTS COMMERCIAUX,10 05 2024,15 05 2024,KOUMASSI (.,1/ Gestion de la base de données clientèle\r\n...,STAGE PEJEDEC,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,KOUMASSI (CMEC KOUMASSI),KOUM-36068-05-2024,3,15/05/2024,BTS,STAGE PEJEDEC,,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
1,STAGIAIRES AGENTS COMMERCIAUX,10 05 2024,15 05 2024,KOUMASSI (.,1/ Gestion de la base de données clientèle\r\n...,STAGE PEJEDEC,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,KOUMASSI (CMEC KOUMASSI),KOUM-36068-05-2024,3,15/05/2024,BTS,STAGE PEJEDEC,,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
2,STAGIAIRES AGENTS COMMERCIAUX,10 05 2024,15 05 2024,KOUMASSI (.,1/ Gestion de la base de données clientèle\r\n...,STAGE PEJEDEC,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,KOUMASSI (CMEC KOUMASSI),KOUM-36068-05-2024,3,15/05/2024,BTS,STAGE PEJEDEC,,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
3,STAGIAIRES AGENTS COMMERCIAUX,10 05 2024,15 05 2024,KOUMASSI (.,1/ Gestion de la base de données clientèle\r\n...,STAGE PEJEDEC,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,KOUMASSI (CMEC KOUMASSI),KOUM-36068-05-2024,3,15/05/2024,BTS,STAGE PEJEDEC,,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
4,STAGIAIRES AGENTS COMMERCIAUX,10 05 2024,15 05 2024,KOUMASSI (.,1/ Gestion de la base de données clientèle\r\n...,STAGE PEJEDEC,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,KOUMASSI (CMEC KOUMASSI),KOUM-36068-05-2024,3,15/05/2024,BTS,STAGE PEJEDEC,,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17211,STAGIAIRE COMMERCIAL,27 02 2024,13 05 2024,,- Rechercher de nouveaux clients;\r\n\r\n- Fid...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABENGOUROU,ABEN-33870-02-2024,2,13/05/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
17212,STAGIAIRE COMMERCIAL,27 02 2024,13 05 2024,,- Rechercher de nouveaux clients;\r\n\r\n- Fid...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABENGOUROU,ABEN-33870-02-2024,2,13/05/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
17213,STAGIAIRE COMMERCIAL,27 02 2024,13 05 2024,,- Rechercher de nouveaux clients;\r\n\r\n- Fid...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABENGOUROU,ABEN-33870-02-2024,2,13/05/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
17214,STAGIAIRE COMMERCIAL,27 02 2024,13 05 2024,,- Rechercher de nouveaux clients;\r\n\r\n- Fid...,STAGE DE QUALIFICATION,Diplôme : ...,https://agenceemploijeunes.ci/site/offres-empl...,,https://agenceemploijeunes.ci/site/offres-empl...,...,ABENGOUROU,ABEN-33870-02-2024,2,13/05/2024,BTS,STAGE DE QUALIFICATION,,BAC+2,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...


In [133]:
7653+17216-8015

16854

In [131]:
df_final = pd.concat([df_final,agenceemploi_jeunes_df], axis=0, ignore_index=True)
df_final
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    if isinstance(text, str):  # Vérifier si le texte est une chaîne de caractères
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
    elif isinstance(text, float):  # Si c'est une valeur float, remplacer par une chaîne vide
        text = ''
    return text

def doublon(df_final):
    # Prétraitement des données
    # Remplacer les valeurs manquantes par une chaîne vide dans les colonnes utilisées
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE'].fillna('')
    df_final['Description_clean'] = df_final['Description'].fillna('')
    df_final['Entreprise_clean'] = df_final['Entreprise'].fillna('')
    df_final['DIPLOME_clean'] = df_final["DIPLOME"].fillna('')
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE"].fillna('')

    # Prétraitement des données
    df_final['Description_clean'] = df_final['Description_clean'].apply(preprocess_text)
    df_final['INTITULE_DU_POSTE_clean'] = df_final['INTITULE_DU_POSTE_clean'].apply(preprocess_text)
    df_final['Entreprise_clean'] = df_final['Entreprise_clean'].apply(preprocess_text)
    df_final['DIPLOME_clean'] = df_final["DIPLOME_clean"].apply(preprocess_text)   
    df_final['TYPE_DE_CONTRAT_DU_POSTE_clean'] = df_final["TYPE_DE_CONTRAT_DU_POSTE_clean"].apply(preprocess_text)

    # Concaténer toutes les variables pour créer un texte combiné
    df_final['Combined_text'] = df_final['INTITULE_DU_POSTE_clean'] + ' ' + df_final['Description_clean'] + ' ' + df_final['Entreprise_clean'] + ' ' + df_final['DIPLOME_clean'] + ' ' + df_final['TYPE_DE_CONTRAT_DU_POSTE_clean']

    # Vectorisation des données
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['Combined_text'])

    # Calcul de similarité
    similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Seuil de similarité
    threshold = 0.9

    # Identification des doublons
    duplicates = set()
    for i in range(len(similarities)):
        for j in range(i+1, len(similarities)):
            if similarities[i,j] > threshold:
                duplicates.add(j)

    # Suppression des doublons
    df_final = df_final.drop(index=duplicates).reset_index(drop=True)

    # Retourner le DataFrame final
    return df_final
df_final=doublon(df_final)
df_final

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,EXPERIENCE_PROFESSIONNELLE,Experience_lettre,URL,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,...,Diploma Requirement,DIPLOME_REQUIS,Job Title_y,Location_y,Reference,Job Type_y,Experience,NIVEAU_D_ETUDES,Gender,Job Description_y
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stagiaire Réseau Fibre Optique et Informatique,WANDOO TECHNOLOGIES,Côte d'ivoire,25 Avril,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,24 Juillet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chiffreur BTP,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Chargé D'Affaires Senior,Exceliam,Côte d'ivoire,08 Février,10,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",08 Mai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assistante Anglais et Français pour un Salon d...,FutureTech,Côte d'ivoire,04 Mars,NaN,Moins d’un an,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Côte d'ivoire,02 Juin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,STAGIAIRE ASSISTANT COMMERCIAL,NaN,NaN,NaN,NaN,NaN,https://agenceemploijeunes.ci/site/offres-empl...,https://agenceemploijeunes.ci/site/offres-empl...,BOUAKE,23/05/2024,...,Diplôme : ...,,STAGIAIRE ASSISTANT COMMERCIAL,BOUAKE,BOUA-34786-03-2024,STAGE PEJEDEC,75.000,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
8011,STAGIAIRE SOIGNANT ET ADMINISTRATIF,NaN,NaN,NaN,NaN,NaN,https://agenceemploijeunes.ci/site/offres-empl...,https://agenceemploijeunes.ci/site/offres-empl...,BOUAKE,23/05/2024,...,Diplôme : ...,,STAGIAIRE ASSISTANT COMMERCIAL,BOUAKE,BOUA-34786-03-2024,STAGE PEJEDEC,75.000,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
8012,AIDE-SOIGNATE,NaN,NaN,NaN,NaN,NaN,https://agenceemploijeunes.ci/site/offres-empl...,https://agenceemploijeunes.ci/site/offres-empl...,ADJAME,23/05/2024,...,Diplôme : ...,,STAGIAIRE ASSISTANT COMMERCIAL,BOUAKE,BOUA-34786-03-2024,STAGE PEJEDEC,75.000,BAC+2,MASCULIN,STAGE PEJEDEC\nDate de clôture:\n ...
8013,STAGIAIRE ASSISTANT COMMERCIAL,NaN,NaN,NaN,NaN,NaN,https://agenceemploijeunes.ci/site/offres-empl...,https://agenceemploijeunes.ci/site/offres-empl...,BOUAKE,27/05/2024,...,Diplôme : ...,,STAGIAIRE SOIGNANT ET ADMINISTRATIF,BOUAKE,BOUA-34441-03-2024,STAGE DE QUALIFICATION,,3EME,MASCULIN,STAGE DE QUALIFICATION\nDate de clôture:\n ...
